# **Comparison between patients OCBS, CSB y HC**

In [1]:
# packages
import numpy as np # numpy package
import pandas as pd # pandaspackages
import matplotlib.pyplot as plt # data visulization
import seaborn as sns # data visulization
import scipy.stats as stats # stats tools
from statsmodels.formula.api import ols
import statsmodels.stats.api as sms
import statsmodels.api as sm
from math import sqrt # square root
from statsmodels.graphics.gofplots import qqplot # qqplot
import pingouin as pg # test welch ANOVA

C:\ProgramData\Anaconda3\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.12, the latest is 0.5.0.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
# functions for the analysis of the data distribution

# mean, standard deviation and CI95 function of two groups
def mean_std(dataA, dataB):
    # counting the number of data, discarding NaN values
    nonNaN_A, nonNaN_B = sum(~np.isnan(dataA)), sum(~np.isnan(dataB))
    # mean and standard deviation
    mean_dataA, std_dataA = np.mean(dataA), np.std(dataA) # dataset A 1
    mean_dataB, std_dataB = np.mean(dataB), np.std(dataB) # dataset B
    # ic 95%
    m95A, M95A = stats.t.interval(0.95, len(dataA) - 1, loc = np.mean(dataA), scale = stats.sem(dataA))
    m95B, M95B = stats.t.interval(0.95, len(dataB) - 1, loc = np.mean(dataB), scale = stats.sem(dataB))
    # returns: sample size (without NaN data), mean and standard deviation of each group
    return nonNaN_A, nonNaN_B, mean_dataA, mean_dataB, std_dataA, std_dataB, m95A, M95A, m95B, M95B


# test normality by Saphiro test function
def normal_dist(dataA, dataB):
    # Saphiro test
    saphiro_stat_dataA, pvalue_saphiro_dataA = stats.shapiro(dataA) # dataset A
    saphiro_stat_dataB, pvalue_saphiro_dataB = stats.shapiro(dataB) # dataset B
    return saphiro_stat_dataA, saphiro_stat_dataB,  pvalue_saphiro_dataA, pvalue_saphiro_dataB # saphirotest stats data

# result report function
def result_report(dataA, dataB):
    # data stats 
    nanA, nanB, mA, mB, sdA, sdB = mean_std(dataA, dataB) # mean and standard deviation of datasets   
    # normality test stat
    statA, statB, pvA, pvB = normal_dist(dataA, dataB)
    return nanA, nanB, mA, mB, sdA, sdB, statA, statB, pvA, pvB

# plot normality
def normal_dist_complete(dataA, dataB):
    # descriptive statistics
    resultados = result_report(dataA, dataB)
    # normality test
    statA, statB, pvA, pvB = normal_dist(dataA, dataB)
    # data zip
    data = pd.DataFrame(zip(dataA, dataB))
    # qqplot
    dataA_qq = qqplot(data[0], line='s') # dataset A
    dataB_qq = qqplot(data[1], line='s') # dataset B
    # distribution plot
    sns.displot(data, kind="kde", rug = True)
    return resultados


# parametric test
# F de Snedecor
def varianza_test(dataA, dataB):
    x = np.array(dataA)
    y = np.array(dataB)
    stat = np.var(dataA, ddof = 1) / np.var(dataB, ddof = 1) # stat F 
    dfn = dataA.size - 1 # defining the number of degrees dataA
    dfd = dataB.size - 1 # defining the number of degrees dataB 
    p = 1 - stats.f.cdf(stat, dfn, dfd) # valorp F test 
    return stat, p


# parametric test contrasting means with equal variances: t de student
def parametric_equalvar(dataA, dataB):
    # name of the test
    definicion = 't de student'
    # contrast test
    stat, p = stats.ttest_ind(dataA, dataB)
    return definicion, stat, p

# parametric test contrasting means with unequal variances: welch's t-test
def parametric_inequalvar(dataA, dataB):
    # name of the test
    definicion = 't de welch'
    # contrast test
    stat, p = stats.ttest_ind(dataA, dataB, equal_var = False)
    return definicion, stat, p

# non-parametric test
# non-parametric contrast test means: 
def non_parametric_two_groups(dataA, dataB):
    # name of the test
    definicion = 'Mann-Whitney U Test'
    # test de Mann-Whitney U Test
    stat, p = stats.mannwhitneyu(dataA, dataB)
    return definicion, stat, p

# defining the table
# data = [[207, 282, 241], [234, 242, 232]]

# stats.fisher_exact(data) # para tamaño muestral grande, sólo 2x2
# stat, p, dof, expected = stats.chi2_contingency(data) # para pequeño

# cohen d test
def cohen(dataA, dataB):
    cd = (np.mean(dataA) - np.mean(dataB)) / (sqrt((np.std(dataA) ** 2 + np.std(dataB) ** 2) / 2))
    return cd



# automated functions
# quantitative variables
def quant_var(dataA, dataB):
    # first objective: does the data have a normal distribution?
    nousedA, nousedB, da, db = normal_dist(dataA, dataB)
    if da > 0.05 and db > 0.05: # in case the saphirowills test pvalues are less than 0.05
        dist = True # normal distribution
    else:
        dist = False # non-normal distribution
    if dist is True: # normal distribution
        # variance
        statf, p = varianza_test(dataA, dataB)
        if p > .05: # if the variance is equal...
            test, stat, p = parametric_equalvar(dataA, dataB) # ...test t student
        else: # if the variance is not equal...
            test, stat, p = parametric_inequalvar(dataA, dataB) # ...test de welch
    else: # non normal...
        test, stat, p = non_parametric_two_groups(dataA, dataB) # ...test MannWhitney U
    return dist, test, stat, p # results returned: type of distribution, test contrast used, test statistic, p-value

def cohen_MW(dataA, dataB, stat):
    cd = (stat) / sqrt((len(dataA) + len(dataB)))
    return cd


# function returning complete result: 
# sample size, mean, variance, CI95, 
# distribution type, contrast test, 
# statistic value, p-value, cohen's d
def analiza(grupo_uno, grupo_dos, dataA, dataB):
    # descriptive analysis
    tamA, tamB, meanA, meanB, sdA, sdB, ic95A, IC95A, ic95B, IC95B = mean_std(dataA, dataB)
    # statistical contrast analysis
    dist, test, stat, p = quant_var(dataA, dataB)
    if test == 't de student' or test == 't de welch':
        # test d de cohen (impacto)
        cd = cohen(dataA, dataB)
    else:
        cd = cohen_MW(dataA, dataB, stat)
    # results
    return [grupo_uno, tamA, meanA, sdA, ic95A, IC95A, grupo_dos, tamB, meanB, sdB, ic95B, IC95B, dist, test, stat, p, cd]

# analysis for 3 groups


# function that returns all the study of a list of variables
def estudio_completo(df, grupo_uno, grupo_dos, var_analisis):
    resultados = []
    for j, i in enumerate(var_analisis):
        # data selection
        grup1 = df[i][df['GRUPO'] == grupo_uno]
        grup2 = df[i][df['GRUPO'] == grupo_dos]
        # data no NaN
        grupoA = grup1[~np.isnan(grup1)]
        grupoB = grup2[~np.isnan(grup2)]
        resultados.append(analiza(grupo_uno, grupo_dos, grupoA, grupoB))
    return resultados


# variables with the names of the result columns
columns_names = ['grupo_uno', 'tamA', 'meanA', 'sdA', 'ic95A', 'IC95A', 
                 'grupo_dos','tamB', 'meanB', 'sdB', 'ic95B', 'IC95B', 
                 'dist', 'tipo de test', 'stat_prueba', 'p-valor', 'd Cohen']



In [3]:
# functions for analysis 3 groups

def mean_std_3g(dataA, dataB, dataC):
    # nonNaN
    nonNaN_A, nonNaN_B, nonNaN_C = sum(~np.isnan(dataA)), sum(~np.isnan(dataB)), sum(~np.isnan(dataC))
    # mean and SD
    mean_dataA, std_dataA = np.mean(dataA), np.std(dataA) # dataset A 
    mean_dataB, std_dataB = np.mean(dataB), np.std(dataB) # dataset B
    mean_dataC, std_dataC = np.mean(dataC), np.std(dataC) # dataset C
    # iq 95%
    m95A, M95A = stats.t.interval(0.95, len(dataA) - 1, loc = np.mean(dataA), scale = stats.sem(dataA))
    m95B, M95B = stats.t.interval(0.95, len(dataB) - 1, loc = np.mean(dataB), scale = stats.sem(dataB))
    m95C, M95C = stats.t.interval(0.95, len(dataC) - 1, loc = np.mean(dataC), scale = stats.sem(dataC))
    # devuelve: tamaño muestra, media y desviación estándar de cada grupo
    return nonNaN_A, mean_dataA, std_dataA, m95A, M95A, nonNaN_B, mean_dataB, std_dataB, m95B, M95B, nonNaN_C, mean_dataC, std_dataC, m95C, M95C


# normality by Saphiro test
def normal_dist_3grupos(dataA, dataB, dataC):
    # Saphiro test
    saphiro_stat_dataA, pvalue_saphiro_dataA = stats.shapiro(dataA) # dataset A
    saphiro_stat_dataB, pvalue_saphiro_dataB = stats.shapiro(dataB) # dataset B
    saphiro_stat_dataC, pvalue_saphiro_dataC = stats.shapiro(dataC) # dataset A
    if pvalue_saphiro_dataA > 0.05 and pvalue_saphiro_dataB > 0.05 and pvalue_saphiro_dataA > 0.05:
        normal = True
    else:
        normal = False
    return normal

# 3-group variance test: Barlelt's test
def varianza_test_3grupos(dataA, dataB, dataC):
    b, p = stats.bartlett(dataA, dataB, dataC)
    if p > 0.05:
        equal_var = True
    else:
        equal_var = False
    return equal_var

# contrast test
# ANOVA (normal distribution, equal variances)
def ANOVA_3grupos(df, grupo, variable):
    # selección de datos
    # escoger datos
    data = df[[grupo, variable]]
    # crear fórmula
    formula = '{} ~ C({})'.format(variable, grupo)
    # crear modelo
    model = ols(formula, data).fit()
    # ANOVA tabla
    aov_table = sm.stats.anova_lm(model, typ=2)
    # resultado 
    return aov_table

# test selectivo para 3 grupos
def test_3g(df, grupo, variable, g1, g2, g3):
    # selección de datos y eliminación NaN
    df_A = df[variable][df[grupo] == g1]
    df_B = df[variable][df[grupo] == g2]
    df_C = df[variable][df[grupo] == g3]
    # análisis descriptivo
    nonNaN_A, mean_dataA, std_dataA, m95A, M95A, nonNaN_B, mean_dataB, std_dataB, m95B, M95B, nonNaN_C, mean_dataC, std_dataC, m95C, M95C = mean_std_3g(df_A, df_B, df_C)
    # primera asunción ANOVA: distribución normal
    normal = normal_dist_3grupos(df_A, df_B, df_C)
    # segunda asunción ANOVA: homogeneidad
    equal_var = varianza_test_3grupos(df_A, df_B, df_C)
    # condiciones para elegir test de contraste
    if normal:
        if equal_var:
            test = 'ANOVA'
            result = ANOVA_3grupos(df, grupo, variable)
            p, r = result['PR(>F)']
        else:
            test = 'Welch ANOVA'
            result = pg.welch_anova(dv = variable, between= grupo, data = df[[grupo, variable]])
            p_b = result['p-unc']
            p = list(p_b)[0]
    else:
        test = 'kruskal-wallis'
        result =  pg.kruskal(dv = variable, between= grupo, data = df[[grupo, variable]])
        p_b = result['p-unc']
        p = list(p_b)[0]
    c = cohen_3g(df_A, df_B, df_C)
    return variable, g1, nonNaN_A, mean_dataA, std_dataA, m95A, M95A, g2, nonNaN_B, mean_dataB, std_dataB, m95B, M95B, g3, nonNaN_C, mean_dataC, std_dataC, m95C, M95C, normal, test, p, c

# test Cohen d 3 grupos
# el test de cohen d
def cohen_3g(dataA, dataB, dataC):
    cd = (np.mean(dataA) - np.mean(dataB) - np.mean(dataC)) / (sqrt((np.std(dataA) ** 2 + np.std(dataB) ** 2 + np.std(dataC) ** 2) / 2))
    return cd

# función estudio completo para tres variables
def estudio_completo_3g(df, grupo, g1, g2, g3, var_analisis):
    resultados = []
    for j, i in enumerate(var_analisis):
        resultados.append(test_3g(df, grupo, i, g1, g2, g3))
    return resultados

# nombres columnas análisis 3 grupos
columns_names2 = ['Variable', 'grupo experimental 1', 'tamaño', 'media', 'SD', 'IC.05', 'IC.95', 
                 'grupo experimental 2', 'tamaño', 'media', 'SD', 'IC.05', 'IC.95',
                  'grupo experimental 3', 'tamaño', 'media', 'SD', 'IC.05', 'IC.95', 'dis', 
                 'tipo de test', 'p-valor', 'd Cohen']

In [4]:
# cargar los datos
df = pd.read_spss('Sex add v Sex Onl 011019 TCI SCL BIS ECS STAIControles_1.sav')

# observar las primeras 10 líneas
df.head()

,CASO,PANHC,PASEX,PAEDA,ORSEX,GRUPO,PAADI03,PAADI05,PANAC,PATRT,...,PSDI,BIScognitiva,BISmotor,BISnoplaneada,TotalBIS,TOTALECS,INTER,FALLOIMP,AE,AR
0,7.0,142473.0,Home,49.0,Heterosexual,Add Sex,sí,no,BCN,si,...,1.364706,16.0,16.0,18.0,50.0,NaN,NaN,NaN,29.0,30.0
1,24.0,142482.0,Home,55.0,Heterosexual,Add Sex,sí,no,bcn,no,...,1.833333,4.0,10.0,12.0,26.0,NaN,NaN,NaN,0.0,4.0
2,30.0,142462.0,Home,34.0,Heterosexual,Add Sex Online,sí,sí,Cataluña. Tarragona,si,...,1.836735,2.0,3.0,13.0,18.0,NaN,NaN,NaN,21.0,16.0
3,37.0,152508.0,Home,20.0,Heterosexual,Add Sex,sí,no,bcn,si,...,1.368421,13.0,13.0,15.0,41.0,NaN,NaN,NaN,34.0,20.0
4,43.0,81822.0,Home,35.0,Homosexual,Add Sex,sí,no,bcn,si,...,1.654545,17.0,23.0,14.0,54.0,27.0,15.0,12.0,25.0,33.0


In [5]:
# significado variables
variables = pd.read_excel('variables definición.xlsx')
variables

,Variable,Descripcion,Código
0,CASO,Numero de Caso,Ninguna
1,PANHC,Numero de Ha Clinica,Ninguna
2,PASEX,Sexe,"{0, Dona}{1, Home}"
3,PAEDA,edad,Ninguna
4,ORSEX,Orientacion Sexual,"{1, Heterosexual}{2, Homosexual}"
5,PANAC,Procedencia,Ninguna
6,PANOC,"si no trabaja, ¿por qué motivo?","{1, paro}{2, baja por enfermedad}{3, jubilado ..."
7,PAECC,estado civil,"{0, soltero}{1, casado/pareja estable}{2, sepa..."
8,PAHIJ,Numero de hijos,Ninguna
9,PATRT,Trabaja?,"{0, no}{1,sí}"


In [6]:
# contar pacientes por grupo 
df['GRUPO'].value_counts()

Add Sex           44
Add Sex Online    36
Control           25
Name: GRUPO, dtype: int64

Vemos que tenemos:

**Pacientes adictos al sexo (ASP):** 44
**Pacientes adictos al cibersexo:** 36
**Controles:** 25

Entre las 59 variables, debemos diferenciar entre las variables que aportan datos no interesan en el análisis, como número de caso e historia clínica. Tras ello, existen 3 grandes grupos: datos demográficos, datos test personalidad y datos test psicopatología.

**Variables test demográficos:**
Sexo, edad, orientación sexual, motivo por el que no trabaja, estado civil, número hijos, ocupado (si trabaja), enfermedades actuales, cuandos años inicio conducta, meses desde que se hizo problemática, alcohol, abuso sustancias, cannabis, cocaína, heroína, alucinógenos, drogas de diseño, otras drogas, tipo inicio conducta, número intendos autolíticos, estudios, si está empleado, años evolución problema del sexo, fumador, edad inicio de la conducta.

Se puede observar que existen variables categóricas y variables cuantitativas. Además existen variables que no tendrán respuesta entre los controles.

Tipo y nombres de las variables en la base de datos:

Variables categóricas demográficas: PASEX, ORSEX, PANOC, PAECC, PATRT, PAENF, PAALC, PASUS, PASUS01, PASUS02, PASUS03, PASUS04, PASUS05, PASUS06, PACAU, PAAUTO, studies, employed, evoluc, tabaco

Variables continuas: PAEDA, PAHIJ, PAANOS, PAAPRO, PAAUTO, evoluc, tabaco, Inicio

**Variables test personalidad:**

TCI pre: Novelty Seeking, TCI pre: Harm Avoidance, TCI pre: Reguard Dependence, TCI pre: Persistence, TCI pre: Self Directness, TCI pre: Cooperation, TCI pre: Autotransendence, TCI pre: validity.

Todas las variables son cuantitativas. Todas engloban a un sólo test, con las siguiente información:

*Temperament and Character Inventory-Revised versión (TCI-R; Cloninger, 1999)*

Cuestionario de 240 ítems con formato Lickert (1 a 5) basado en el cuestionario TCI (Cloninger, 1993). Mide 7 dimensiones de personalidad: temperamentales (Evitación del Daño, Dependencia a la Recompensa, Búsqueda de la Novedad, Persistencia) y caracteriales (Autodirección, Cooperación y Autotrascendencia). Su validación y adaptación española ha demostrado buenas propiedades psicométricas con coeficientes alpha de Cronbach entre 0.77 y 0.84 (Gutierrez-Zotes et al., 2004)

Nombres de las variables según la base de datos: NSTOTAL, HATOTAL, RDTOTAL, PSTOTAL, SDTOTAL, CTOTAL, STTOTAL, VALID.

**Variables de test psicopatología:**

Pre SCL-90: Somatización, Pre SCL-90: Obsesión-Compulsión, Pre SCL-90: Sensibilidad Interpersonal, Pre SCL-90: Depresion, Pre SCL-90: Ansiedad, Pre SCL-90: Hostilidad, Pre SCL-90: Fobia, Pre SCL-90: Paranoia, Pre SCL-90: Psicoticismo, Pre SCL-90: Severidad Global, Pre SCL-90: PSDI, Pre BIS: Imp Cognitiva, Pre BIS: Imp motora, Pre BIS: Imp No Planeada, Pre BIS: Imp TOTAL, Pre ECS: Add Sexual: Total, Pre ECS: Add Sexual: Interferencia, Pre ECS: Add Sexual: Fallo control impulsos, Pre STAI: Ans Estado, Pre STAI: Ans Rasgo.

Todas las variables son cuantitativas. Aquí existen hasta 5 tipos de test diferentes. Esta es la información sobre cada uno de ellos y sus variables:

*Symptom CheckList-90 items-Revised (Derogatis, 1990)*

Cuestionario ampliamente utilizado para medir el estado psicopatológico actual (Derogatis, 1990). Consiste en 90 ítems con formato tipo Lickert (5 puntos) que representan diversos grados de intensidad del síntoma en cuestión (Derogatis, 1983). Da lugar a 9 dimensiones sintomáticas primarias: Somatización, Obsesivo-Compulsiva, Sensibilidad Interpersonal, Hostilidad, Depresión, Ansiedad, Ansiedad Fóbica, Psicoticismo e Ideación Paranoide. También permite valorar el estado psicopatológico mediante 3 indices globales de funcionamiento (Positive Symptom Total, Global SeverityIndex y General SymptomDistressIndex). Esta escala está validada en España con buenas propiedades psicométricas (Derogatis, 2002).

Variables: Pre SCL-90: Somatización, Pre SCL-90: Obsesión-Compulsión, Pre SCL-90: Sensibilidad Interpersonal, Pre SCL-90: Depresion, Pre SCL-90: Ansiedad, Pre SCL-90: Hostilidad, Pre SCL-90: Fobia, Pre SCL-90: Paranoia, Pre SCL-90: Psicoticismo, Pre SCL-90: Severidad Global, Pre SCL-90: PSDI

*The Rosenberg Self-Esteem Scale (RSES) (Rosenberg, 1989)*

Es una escala utilizada para valorar el nivel de autoestima (Test de Rosenberg). La prueba consta de 10 preguntas, puntuables entre 1 y 4 puntos, lo que permite obtener una puntuación mínima de 10 y máxima de 40. Esta escala está validad en España con buenas propiedades psicométricas (Vázquez, Jiménez, Vázquez-Morejón, 2004).

Variables: [**faltan por identificar**]

*State-Trait Anxiety Inventory (STAI) (Spielberger, Gorsuch, &Lushene, 1970)*

Cuestionario que evalúa el nivel actual de ansiedad (AE) y la predisposición de la persona a responder al estrés (AR), dos escalas en total. Consta de 40 preguntas, la puntuación para cada escala puede oscilar de 0-30, indicando las puntuaciones más altas, mayores niveles de ansiedad.Esta escala está validad en España con El cuestionario ha sido validad en España y ha mostrado una buena consistencia interna de entre 0.90 y 0.94. (Guillén-Riquelme y Buela-Casal 2011).

Variables: Pre STAI: Ans Estado, Pre STAI: Ans Rasgo

*The Barratt Impulsiveness Scale (BIS-11) (Patton et al., 1995)*

Es un cuestionario diseñado para medir el constructo de la impulsividad personalidad/comportamiento. Dispone de cuatro escalas: Impulsividad Cognitiva, Impulsividad Motora, Impulsividad No planeada, Impulsividad Total. Está traducido y adaptado al Español (Oquendo, Baca-Garcia, Graver, Morales, Montalvan & Mann, 2001)

Variables: Pre BIS: Imp Cognitiva, Pre BIS: Imp motora, Pre BIS: Imp No Planeada, Pre BIS: Imp TOTAL

*Escala de Compulsividad Sexual (ECS) (Kalichman et al., 1994)*

Cuestionario de 10 ítems que evalúan la compulsividad sexual. Dispone de tres escalas: Interferencia, Fallo al controlar los impulsos sexuales y puntuación total. Esta escala está validad en España con buenas propiedades psicométricas, la fiabilidad y estabilidad temporal de esta versión fue de 0.83 y 0.72. (Ballester-Arnal et al., 2013).

Variables: Pre ECS: Add Sexual: Total, Pre ECS: Add Sexual: Interferencia, Pre ECS: Add Sexual: Fallo control impulsos

Nombres variables según base de datos: TOTALEC,	INTER, FALLOIMP.	

In [7]:
# nombres columnas base de datos
df.columns

Index(['CASO', 'PANHC', 'PASEX', 'PAEDA', 'ORSEX', 'GRUPO', 'PAADI03',
       'PAADI05', 'PANAC', 'PATRT', 'PANOC', 'PAECC', 'PAHIJ', 'PAENF',
       'PATRE', 'PAANOS', 'PAAPRO', 'PAALC', 'PASUS', 'PASUS01', 'PASUS02',
       'PASUS03', 'PASUS04', 'PASUS05', 'PASUS06', 'PACAU', 'PAAUTO',
       'studies', 'employed', 'evoluc', 'tabaco', 'Inicio', 'NSTOTAL',
       'HATOTAL', 'RDTOTAL', 'PSTOTAL', 'SDTOTAL', 'CTOTAL', 'STTOTAL',
       'VALID', 'SOMATP', 'OBSESP', 'SENSIP', 'DEPRESP', 'ANSIEP', 'HOSTILP',
       'FOBIAP', 'PARANP', 'PSICOTP', 'GSI', 'PSDI', 'BIScognitiva',
       'BISmotor', 'BISnoplaneada', 'TotalBIS', 'TOTALECS', 'INTER',
       'FALLOIMP', 'AE', 'AR'],
      dtype='object')

# Comparación entre patientes: ASP vs ASO

## Demografía

Aquí nos encontramos variables que son cuantitativas, que se analizarán con las funciones automatizadas, y variables cualitativas, donde se realizará una tabla de contingencia y el análisis correspondiente.

Cuando las frecuencias teóricas incluyen frecuencias que son inferiores a 5, o cuando las sumas marginales del conjunto de datos (sumas por fila o por columna) son muy desiguales, es mejor confiar en la prueba exacta de Fisher.

### Variables Cualitativas

In [8]:
# variable para guardar los resultados
resultados_cualitativa = []

# escoger datos
# tabla contingencia 'ORSEX'
df2 = df.copy()

data = df2[(df['GRUPO'] == 'Add Sex') | (df['GRUPO'] == 'Add Sex Online')]

**Orientación sexual**

In [9]:
# por orientación sexual: 'ORSEX'
cross = pd.crosstab(data['GRUPO'], 
            data['ORSEX'], margins = False)
cross

ORSEX,Bisexual,Heterosexual,Homosexual
GRUPO,,,
Add Sex,2,29,13
Add Sex Online,1,33,2


In [10]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}    dof: {}'.format(stat, p, dof))

resultados_cualitativa.append(['Orientación sexual', 'chi2',stat, p])

Stat: 7.937438905180842   p-valor: 0.01889761694201347    dof: 2


In [11]:
# por orientación sexual: 'PACAU'
cross = pd.crosstab(data['GRUPO'], 
            data['PACAU'], margins = False)
cross

PACAU,acontecimiento vital desencadenante,casual/distracción,socio-familiar
GRUPO,,,
Add Sex,11,31,2
Add Sex Online,6,28,2


In [12]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}    dof: {}'.format(stat, p, dof))

resultados_cualitativa.append(['Orientación sexual', 'chi2',stat, p])

Stat: 0.831445058763105   p-valor: 0.6598633377737404    dof: 2


### Si no trabaja, ¿Por qué motivo?

In [13]:
# tabla contingencia 'PANOC'
cross = pd.crosstab(data['GRUPO'], 
            data['PANOC'], margins = False)
cross

PANOC,baja por enfermedad,jubilado (incluye anticipada),no ocupación,paro
GRUPO,,,,
Add Sex,3,5,1,4
Add Sex Online,3,2,1,2


In [14]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}    dof: {}'.format(stat, p, dof))

resultados_cualitativa.append(['motivo parado', 'chi2', stat, p])

Stat: 0.8076923076923078   p-valor: 0.8476262639524308    dof: 3


### Estado civil

In [15]:
# tabla contingencia 'PANOC'
cross = pd.crosstab(data['GRUPO'], 
            data['PAECC'], margins = False)
cross

PAECC,casado/pareja estable,separado/divorciado,soltero,viudo
GRUPO,,,,
Add Sex,25,6,12,1
Add Sex Online,27,5,4,0


In [16]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}    dof: {}'.format(stat, p, dof))

resultados_cualitativa.append(['Estado civil', 'chi2', stat, p])

Stat: 4.411951684678957   p-valor: 0.2202797438093587    dof: 3


### ¿Tiene hijos o no?

In [17]:
# preparación de datos
boolerian = []
a = 0
for i in data[['GRUPO', 'PAHIJ']]['PAHIJ']:
    if i > 0:
        boolerian.append('Si')
    if i == 0:
        boolerian.append('No')
    else:
        a += 1

newdata = pd.DataFrame(zip(data[['GRUPO', 'PAHIJ']]['GRUPO'], boolerian))

# tabla contigencia
cross = pd.crosstab(newdata[0], 
            newdata[1], margins = False)

cross

1,No,Si
0,,
Add Sex,23,20
Add Sex Online,15,21


In [18]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}    dof: {}'.format(stat, p, dof))

resultados_cualitativa.append(['¿Hijos?', 'chi2', stat, p])

Stat: 0.674516353869169   p-valor: 0.41148141795323323    dof: 1


### Trabaja

In [19]:
# tabla contigencia 'PATRT'
cross = pd.crosstab(data['GRUPO'], 
            data['PATRT'], margins = False)
cross

PATRT,no,si
GRUPO,,
Add Sex,14,30
Add Sex Online,11,25


In [20]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}    dof: {}'.format(stat, p, dof))

resultados_cualitativa.append(['¿Trabaja?', 'chi2', stat, p])

Stat: 0.014692378328741965   p-valor: 0.9035230235700373    dof: 1


### Enfermedades actuales

In [21]:
# tabla contigencia 'PAENF'
cross = pd.crosstab(data['GRUPO'], 
            data['PAENF'], margins = False)
cross

PAENF,no,si
GRUPO,,
Add Sex,23,20
Add Sex Online,30,6


In [22]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}    dof: {}'.format(stat, p, dof))

resultados_cualitativa.append(['¿Otra enfermedad?', 'chi2', stat, p])

Stat: 6.610896287358002   p-valor: 0.010135663457800852    dof: 1


### Presenta abuso de alcohol

In [23]:
# tabla contigencia 'PAALC'
cross = pd.crosstab(data['GRUPO'], 
            data['PAALC'], margins = False)
cross

PAALC,no,sí
GRUPO,,
Add Sex,41,3
Add Sex Online,35,1


In [24]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}    dof: {}'.format(stat, p, dof))

resultados_cualitativa.append(['¿Abuso alcohol?', 'chi2', stat, p])

Stat: 0.0956937799043061   p-valor: 0.7570597784984473    dof: 1


### Presenta abuso de sustancias

In [25]:
# tabla contigencia 'PASUS'
cross = pd.crosstab(data['GRUPO'], 
            data['PASUS'], margins = False)
cross

PASUS,no,sí
GRUPO,,
Add Sex,35,9
Add Sex Online,31,5


In [26]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}    dof: {}'.format(stat, p, dof))

resultados_cualitativa.append(['¿Abuso sustancias?', 'chi2', stat, p])

Stat: 0.22388386024749646   p-valor: 0.6360964136112718    dof: 1


### ¿Abuso cannabis?

In [27]:
# tabla contigencia 'PASUS01'
cross = pd.crosstab(data['GRUPO'], 
            data['PASUS01'], margins = False)
cross

PASUS01,no,sí
GRUPO,,
Add Sex,37,7
Add Sex Online,31,5


In [28]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}    dof: {}'.format(stat, p, dof))

resultados_cualitativa.append(['¿Abuso cannabis?', 'chi2', stat, p])

Stat: 0.00396118043176866   p-valor: 0.9498159473833612    dof: 1


### ¿Abuso cocaína?

In [29]:
# tabla contigencia 'PASUS02'
cross = pd.crosstab(data['GRUPO'], 
            data['PASUS02'], margins = False)
cross

PASUS02,no,sí
GRUPO,,
Add Sex,39,5
Add Sex Online,36,0


In [30]:
# test chi cuadrado
oddsratio, p = stats.fisher_exact(cross)

print('Stat: {}   p-valor: {}    dof: {}'.format(stat, p, dof))

resultados_cualitativa.append(['¿Abuso cocaína?', 'fisher', oddsratio, p])

Stat: 0.00396118043176866   p-valor: 0.06085686465433306    dof: 1


### ¿Abuso heroína?

In [31]:
# tabla contigencia 'PASUS03'
cross = pd.crosstab(data['GRUPO'], 
            data['PASUS03'], margins = False)
cross

PASUS03,no
GRUPO,
Add Sex,44
Add Sex Online,36


Sin consumo entre pacientes

### ¿Abuso alucinógenos?

In [32]:
# tabla contigencia 'PASUS04'
cross = pd.crosstab(data['GRUPO'], 
            data['PASUS04'], margins = False)
cross

PASUS04,no,sí
GRUPO,,
Add Sex,43,1
Add Sex Online,36,0


In [33]:
# test chi cuadrado
oddsratio, p = stats.fisher_exact(cross)

print('Stat: {}   p-valor: {}    dof: {}'.format(stat, p, dof))

resultados_cualitativa.append(['¿Abuso alucinógenos?', 'fisher', oddsratio, p])

Stat: 0.00396118043176866   p-valor: 1.0    dof: 1


### ¿Abuso drogas de diseño?

In [34]:
# tabla contigencia 'PASUS05'
cross = pd.crosstab(data['GRUPO'], 
            data['PASUS05'], margins = False)
cross

PASUS05,no,sí
GRUPO,,
Add Sex,42,2
Add Sex Online,36,0


In [35]:
# test chi cuadrado
oddsratio, p = stats.fisher_exact(cross)

print('Stat: {}   p-valor: {}    dof: {}'.format(stat, p, dof))

resultados_cualitativa.append(['¿Abuso drogas de diseño?', 'fisher', oddsratio, p])

Stat: 0.00396118043176866   p-valor: 0.4987341772151902    dof: 1


### ¿Abuso de otras drogas?

In [36]:
# tabla contigencia 'PASUS06'
cross = pd.crosstab(data['GRUPO'], 
            data['PASUS06'], margins = False)
cross

PASUS06,no,sí
GRUPO,,
Add Sex,42,2
Add Sex Online,36,0


In [37]:
# test chi cuadrado
oddsratio, p = stats.fisher_exact(cross)

print('Stat: {}   p-valor: {}    dof: {}'.format(stat, p, dof))

resultados_cualitativa.append(['¿Abuso de otras drogas?', 'fisher', oddsratio, p])

Stat: 0.00396118043176866   p-valor: 0.4987341772151902    dof: 1


### ¿Ha habido intentos de autolíticos?

In [38]:
# preparación de datos 'PAAUTO'
boolerian = []
a = 0
for i in data[['GRUPO', 'PAAUTO']]['PAAUTO']:
    if i > 0:
        boolerian.append('Si')
    if i == 0:
        boolerian.append('No')
    else:
        a += 1

newdata = pd.DataFrame(zip(data[['GRUPO', 'PAAUTO']]['GRUPO'], boolerian))

# tabla contigencia
cross = pd.crosstab(newdata[0], 
            newdata[1], margins = False)

cross

1,No,Si
0,,
Add Sex,40,4
Add Sex Online,32,4


In [39]:
# test fisher
oddsratio, p = stats.fisher_exact(cross)

print('Stat: {}   p-valor: {}'.format(oddsratio, p))

resultados_cualitativa.append(['¿Ha habido intentos de autolíticos?', 'fisher', oddsratio, p])

Stat: 1.25   p-valor: 1.0


### Nivel de estudios

In [40]:
# tabla contigencia 'studies'
cross = pd.crosstab(data['GRUPO'], 
            data['studies'], margins = False)
cross

studies,No estudios,Primarios,Secundarios,Universitarios
GRUPO,,,,
Add Sex,1,15,11,17
Add Sex Online,0,11,9,16


In [41]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}    dof: {}'.format(stat, p, dof))

resultados_cualitativa.append(['Nivel de estudios', 'chi2', stat, p])

Stat: 1.0562501471592376   p-valor: 0.7876445584129561    dof: 3


### ¿Tiene empleo?

In [42]:
# tabla contigencia 'employed'
cross = pd.crosstab(data['GRUPO'], 
            data['employed'], margins = False)
cross

employed,No,Sí
GRUPO,,
Add Sex,14,30
Add Sex Online,9,27


In [43]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}    dof: {}'.format(stat, p, dof))

resultados_cualitativa.append(['¿Tiene empleo?', 'chi2', stat, p])

Stat: 0.17813528111010954   p-valor: 0.6729806721328858    dof: 1


### ¿Es fumador?

In [44]:
# tabla contigencia 'tabaco'
cross = pd.crosstab(data['GRUPO'], 
            data['tabaco'], margins = False)
cross

tabaco,No,Sí
GRUPO,,
Add Sex,31,13
Add Sex Online,26,10


In [45]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}    dof: {}'.format(stat, p, dof))

resultados_cualitativa.append(['¿Es fumador?', 'chi2', stat, p])

Stat: 0.005547465501698899   p-valor: 0.9406274125156404    dof: 1


**Resultados variables cualitativas.**

In [46]:
output = pd.DataFrame(resultados_cualitativa, columns = ['Variable', 'Prueba', 'Stat', 'p-valor'])

output.to_excel("ASPvsASO_demografía_variables_cualitativas.xlsx")
output

,Variable,Prueba,Stat,p-valor
0,Orientación sexual,chi2,7.937439,0.018898
1,Orientación sexual,chi2,0.831445,0.659863
2,motivo parado,chi2,0.807692,0.847626
3,Estado civil,chi2,4.411952,0.220280
4,¿Hijos?,chi2,0.674516,0.411481
5,¿Trabaja?,chi2,0.014692,0.903523
6,¿Otra enfermedad?,chi2,6.610896,0.010136
7,¿Abuso alcohol?,chi2,0.095694,0.757060
8,¿Abuso sustancias?,chi2,0.223884,0.636096
9,¿Abuso cannabis?,chi2,0.003961,0.949816


Se observan diferencias estadisticamente significativas en las variables: orientación sexual y presenta otras enfermedades.

### Variables cuantitativas

In [47]:
# mostras datos variable
df[['PAEDA', 'PAHIJ', 'PAANOS', 'PAAPRO', 'PAAUTO', 'evoluc', 'Inicio']]

,PAEDA,PAHIJ,PAANOS,PAAPRO,PAAUTO,evoluc,Inicio
0,49.0,3.0,4.0,24.0,0.0,2.000000,47.000000
1,55.0,1.0,1.0,12.0,0.0,1.000000,54.000000
2,34.0,0.0,10.0,20.0,0.0,1.666667,32.333333
3,20.0,0.0,10.0,60.0,0.0,5.000000,15.000000
4,35.0,0.0,17.0,72.0,0.0,6.000000,29.000000
...,...,...,...,...,...,...,...
100,61.0,2.0,0.0,0.0,0.0,0.000000,0.000000
101,26.0,0.0,0.0,0.0,0.0,0.000000,0.000000
102,30.0,0.0,0.0,0.0,0.0,0.000000,0.000000
103,30.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [48]:
# variables a analizar
var_analisis = ['PAEDA', 'PAHIJ', 'PAANOS', 'PAAPRO', 'PAAUTO', 'evoluc', 'Inicio']

output = pd.DataFrame(estudio_completo(df, 'Add Sex', 'Add Sex Online', var_analisis), 
             index = var_analisis, columns = columns_names)

output.to_excel("ASPvsASO_demografía_variables_cuantitativas.xlsx")
output

,grupo_uno,tamA,meanA,sdA,ic95A,IC95A,grupo_dos,tamB,meanB,sdB,ic95B,IC95B,dist,tipo de test,stat_prueba,p-valor,d Cohen
PAEDA,Add Sex,44,43.454545,11.947717,39.780113,47.128978,Add Sex Online,36,42.250000,10.003819,38.817181,45.682819,True,t de student,0.476150,0.635298,0.109318
PAHIJ,Add Sex,44,0.863636,1.035526,0.545168,1.182105,Add Sex Online,35,1.000000,1.014185,0.646529,1.353471,False,Mann-Whitney U Test,700.500000,0.232413,78.812407
PAANOS,Add Sex,44,15.227273,9.876196,12.189921,18.264625,Add Sex Online,36,13.555556,9.607417,10.258762,16.852349,False,Mann-Whitney U Test,708.000000,0.209243,79.156806
PAAPRO,Add Sex,44,92.022727,96.568037,62.323935,121.721520,Add Sex Online,36,75.250000,88.927991,44.734283,105.765717,False,Mann-Whitney U Test,717.000000,0.234965,80.163037
PAAUTO,Add Sex,44,0.113636,0.382332,-0.003947,0.231220,Add Sex Online,36,0.111111,0.314270,0.003269,0.218953,False,Mann-Whitney U Test,778.000000,0.400873,86.983044
evoluc,Add Sex,44,7.668561,8.047336,5.193661,10.143460,Add Sex Online,36,6.270833,7.410666,3.727857,8.813810,False,Mann-Whitney U Test,717.000000,0.234965,80.163037
Inicio,Add Sex,44,36.013258,14.807412,31.459346,40.567169,Add Sex Online,36,35.979167,11.985530,31.866322,40.092012,True,t de student,0.011006,0.991247,0.002531


En este análisis, hay que tener en cuenta que la mayoría de los pacientes no tiene hijos ni se ha infringido autolesiones, por lo que estas variables no son correctas para ser analizadas como variables cuantitativas. Serán analizadas por cualitativas, dejando en sí o no.

## Personalidad

**Las variables que se van a comparar son las siguientes**: TCI pre: Novelty Seeking, TCI pre: Harm Avoidance, TCI pre: Reguard Dependence, TCI pre: Persistence, TCI pre: Self Directness, TCI pre: Cooperation, TCI pre: Autotransendence, TCI pre: validity

In [49]:
# datos de la personalidad
df[['GRUPO', 'NSTOTAL', 'HATOTAL', 
    'RDTOTAL', 'PSTOTAL', 
    'SDTOTAL', 'CTOTAL', 
    'STTOTAL', 'VALID']][(df['GRUPO'] == 'Add Sex') | (df['GRUPO'] == 'Add Sex Online') ]

,GRUPO,NSTOTAL,HATOTAL,RDTOTAL,PSTOTAL,SDTOTAL,CTOTAL,STTOTAL,VALID
0,Add Sex,101.0,106.0,99.0,124.0,130.0,137.0,71.0,15.0
1,Add Sex,107.0,73.0,114.0,151.0,176.0,148.0,50.0,15.0
2,Add Sex Online,81.0,91.0,125.0,116.0,171.0,159.0,75.0,15.0
3,Add Sex,95.0,110.0,88.0,97.0,146.0,132.0,85.0,15.0
4,Add Sex,102.0,137.0,89.0,111.0,116.0,113.0,48.0,15.0
...,...,...,...,...,...,...,...,...,...
75,Add Sex Online,125.0,92.0,93.0,98.0,119.0,130.0,50.0,15.0
76,Add Sex Online,86.0,126.0,81.0,90.0,111.0,125.0,53.0,15.0
77,Add Sex Online,84.0,117.0,88.0,85.0,136.0,140.0,51.0,15.0
78,Add Sex Online,137.0,82.0,102.0,114.0,82.0,105.0,56.0,15.0


In [50]:
# análisis items de test personalidad
var_analisis = ['NSTOTAL', 'HATOTAL', 
                'RDTOTAL', 'PSTOTAL', 
                'SDTOTAL', 'CTOTAL', 
                'STTOTAL', 'VALID']


output = pd.DataFrame(estudio_completo(df, 'Add Sex', 'Add Sex Online', var_analisis), 
             index = var_analisis, columns = columns_names)

output.to_excel("ASPvsASO_personalidad.xlsx")
output

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:2098: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:2099: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


,grupo_uno,tamA,meanA,sdA,ic95A,IC95A,grupo_dos,tamB,meanB,sdB,ic95B,IC95B,dist,tipo de test,stat_prueba,p-valor,d Cohen
NSTOTAL,Add Sex,44,107.772727,13.654609,103.573352,111.972102,Add Sex Online,36,102.166667,17.926548,96.015156,108.318177,True,t de student,1.566767,0.121217,0.351822
HATOTAL,Add Sex,44,104.272727,21.076014,97.790954,110.754501,Add Sex Online,36,105.111111,21.743170,97.649923,112.572299,True,t de student,-0.172304,0.863645,-0.039155
RDTOTAL,Add Sex,44,99.704545,15.187584,95.033715,104.375376,Add Sex Online,36,98.416667,12.846043,94.008536,102.824798,True,t de student,0.399005,0.690980,0.091562
PSTOTAL,Add Sex,44,108.295455,18.798670,102.514061,114.076848,Add Sex Online,36,106.055556,20.993312,98.851682,113.259429,True,t de student,0.496637,0.620842,0.112410
SDTOTAL,Add Sex,44,124.181818,24.937442,116.512490,131.851146,Add Sex Online,36,125.611111,20.491567,118.579412,132.642810,True,t de student,-0.272529,0.785935,-0.062625
CTOTAL,Add Sex,44,129.818182,21.413964,123.232474,136.403890,Add Sex Online,36,133.416667,14.595233,128.408300,138.425034,False,Mann-Whitney U Test,756.000000,0.365622,84.523370
STTOTAL,Add Sex,44,67.386364,14.851103,62.819016,71.953712,Add Sex Online,36,67.027778,15.507142,61.706489,72.349067,False,Mann-Whitney U Test,769.000000,0.413831,85.976814
VALID,Add Sex,44,15.000000,0.000000,NaN,NaN,Add Sex Online,36,14.972222,0.371641,14.844693,15.099751,False,Mann-Whitney U Test,792.000000,0.492868,88.548292


<font size= 1.5, color=red>**NOTA**: El error de devuelve python es debido a que el ítem **VALID** tiene valor 15 para todos los pacientes, a excepción de un paciente adicto al sexo online, que tiene valor 13. El error es consecuencia del cálculo de los índices de confianza 95% de los adictos al sexo presencial, ya que todos sus valores son 15 y no es posible calcular el IC95.</font>

Desde el punto de vista del análisis, podemos observar que ninguna de las variables de la personalidad presenta diferencias, ya que sus valores-p son muy similares.

## Psicopatología

Análisis las variables que pertenecen a **Symptom CheckList-90 items-Revised (Derogatis, 1990)**

In [51]:
# análisis items 'Symptom CheckList-90 items-Revised (Derogatis, 1990)'
var_analisis = ['SOMATP', 'OBSESP', 'SENSIP', 'DEPRESP', 
                'ANSIEP', 'HOSTILP', 'FOBIAP', 'PARANP', 
                'PSICOTP', 'GSI', 'PSDI']

output = pd.DataFrame(estudio_completo(df, 'Add Sex', 'Add Sex Online', var_analisis), 
             index = var_analisis, columns = columns_names)

output.to_excel("ASPvsASO_psicopatologia_SCL.xlsx")
output

,grupo_uno,tamA,meanA,sdA,ic95A,IC95A,grupo_dos,tamB,meanB,sdB,ic95B,IC95B,dist,tipo de test,stat_prueba,p-valor,d Cohen
SOMATP,Add Sex,44,1.047348,0.845815,0.787224,1.307473,Add Sex Online,36,0.930556,0.620664,0.717574,1.143537,False,Mann-Whitney U Test,759.500000,0.378298,84.914681
OBSESP,Add Sex,44,1.502273,0.794152,1.258037,1.746508,Add Sex Online,36,1.452778,0.776204,1.186423,1.719133,True,t de student,0.276633,0.782794,0.063033
SENSIP,Add Sex,44,1.371212,0.998580,1.064106,1.678318,Add Sex Online,36,1.157407,0.790624,0.886104,1.428711,False,Mann-Whitney U Test,702.500000,0.194462,78.541888
DEPRESP,Add Sex,44,1.617133,0.948582,1.325403,1.908862,Add Sex Online,36,1.478632,0.796960,1.205155,1.752110,False,Mann-Whitney U Test,737.500000,0.300589,82.455007
ANSIEP,Add Sex,44,1.331818,0.895893,1.056293,1.607343,Add Sex Online,36,0.886111,0.650279,0.662967,1.109255,False,Mann-Whitney U Test,576.500000,0.018622,64.454659
HOSTILP,Add Sex,44,1.208333,1.051169,0.885054,1.531613,Add Sex Online,36,0.782407,0.651233,0.558936,1.005878,False,Mann-Whitney U Test,626.000000,0.054009,69.988928
FOBIAP,Add Sex,44,0.639610,0.808976,0.390816,0.888405,Add Sex Online,36,0.607143,0.738767,0.353634,0.860651,False,Mann-Whitney U Test,764.500000,0.395413,85.473698
PARANP,Add Sex,44,1.250000,0.879408,0.979544,1.520456,Add Sex Online,36,0.967593,0.759527,0.706960,1.228225,True,t de student,1.499282,0.137837,0.343705
PSICOTP,Add Sex,44,1.377273,0.866514,1.110783,1.643763,Add Sex Online,36,1.072222,0.656849,0.846824,1.297621,True,t de welch,1.767445,0.081084,0.396756
GSI,Add Sex,44,1.318434,0.784766,1.077085,1.559783,Add Sex Online,36,1.105556,0.569248,0.910218,1.300893,True,t de welch,1.386291,0.169658,0.310532


Desde el punto de vista del análsis, podemos observar que existen diferencias estadísticamente significativas entre los pacientes en el ítem **Pre SCL-90: Ansiedad**, donde se muesta un pvalor < 0.05 (0.018).

- Media ASP: 1.33 (±0.896)

- Media ASO: 0.886 (±0.65)

Análisis de las variables que pertenecen a **El Inventario de Ansiedad Estado-Rasgo (STAI)**.

In [52]:
# análisis items 'El Inventario de Ansiedad Estado-Rasgo (STAI)'
var_analisis = ['AE', 'AR']

output = pd.DataFrame(estudio_completo(df, 'Add Sex', 'Add Sex Online', var_analisis), 
             index = var_analisis, columns = columns_names)

output.to_excel("ASPvsASO_psicopatologia_STAI.xlsx")
output

,grupo_uno,tamA,meanA,sdA,ic95A,IC95A,grupo_dos,tamB,meanB,sdB,ic95B,IC95B,dist,tipo de test,stat_prueba,p-valor,d Cohen
AE,Add Sex,44,24.863636,14.554898,20.387384,29.339888,Add Sex Online,36,22.916667,9.633204,19.611024,26.222309,True,t de welch,0.707264,0.481593,0.157753
AR,Add Sex,44,29.363636,11.004319,25.979339,32.747934,Add Sex Online,36,26.277778,11.059463,22.482713,30.072842,True,t de student,1.229329,0.222644,0.279721


No se aprecian diferencias significativas entre pacientes ASP y ASO en las variables del test STAI.

Análisis de plas variables de **The Barratt Impulsiveness Scale (BIS-11) (Patton et al., 1995)**

In [53]:
# análisis items 'The Barratt Impulsiveness Scale (BIS-11)'
var_analisis = ['BIScognitiva', 'BISmotor', 'BISnoplaneada', 'TotalBIS']

output = pd.DataFrame(estudio_completo(df, 'Add Sex', 'Add Sex Online', var_analisis), 
             index = var_analisis, columns = columns_names)

output.to_excel("ASPvsASO_psicopatologia_BIS11.xlsx")
output

,grupo_uno,tamA,meanA,sdA,ic95A,IC95A,grupo_dos,tamB,meanB,sdB,ic95B,IC95B,dist,tipo de test,stat_prueba,p-valor,d Cohen
BIScognitiva,Add Sex,44,16.750000,5.257570,15.133073,18.366927,Add Sex Online,36,16.722222,5.495509,14.836434,18.608011,True,t de student,0.022745,0.981912,0.005165
BISmotor,Add Sex,44,19.000000,6.928203,16.869282,21.130718,Add Sex Online,36,15.833333,7.661230,13.204376,18.462291,True,t de student,1.914560,0.059216,0.433557
BISnoplaneada,Add Sex,44,19.909091,7.558313,17.584587,22.233595,Add Sex Online,36,21.111111,8.468168,18.205252,24.016970,True,t de student,-0.661777,0.510065,-0.149763
TotalBIS,Add Sex,44,55.659091,14.001531,51.353023,59.965159,Add Sex Online,36,53.666667,17.603661,47.625955,59.707378,True,t de student,0.556709,0.579320,0.125271


No se observan diferencias entre pacientes en ninguno de los ítems analizados de BIS11.

Análisis de plas variables de **Escala de Compulsividad Sexual (SCS) (Kalichman et al., 1994)**.

In [54]:
# análisis items 'Escala de Compulsividad Sexual (SCS)'
var_analisis = ['TOTALECS', 'INTER', 'FALLOIMP']

output = pd.DataFrame(estudio_completo(df, 'Add Sex', 'Add Sex Online', var_analisis), 
             index = var_analisis, columns = columns_names)

output.to_excel("ASPvsASO_psicopatologia_SCS.xlsx")
output

,grupo_uno,tamA,meanA,sdA,ic95A,IC95A,grupo_dos,tamB,meanB,sdB,ic95B,IC95B,dist,tipo de test,stat_prueba,p-valor,d Cohen
TOTALECS,Add Sex,40,28.750,7.234466,26.406830,31.093170,Add Sex Online,34,25.676471,6.474932,23.383285,27.969656,True,t de student,1.884739,0.063501,0.447696
INTER,Add Sex,40,14.175,3.885148,12.916640,15.433360,Add Sex Online,34,12.941176,3.262453,11.785734,14.096619,True,t de student,1.444313,0.152989,0.343939
FALLOIMP,Add Sex,40,15.125,4.166458,13.775527,16.474473,Add Sex Online,34,12.764706,3.896143,11.384834,14.144578,True,t de student,2.467771,0.015973,0.585163


Desde el punto de vista del análsis, podemos observar que existen diferencias estadísticamente significativas entre los pacientes en el ítem **Pre ECS: Add Sexual: Fallo control impulsos**, donde se muesta un pvalor < 0.05 (0.016).

- Media ASP: 15.12 (±4.17)

- Media ASO: 12.76 (±3.9)

# Comparación entre patientes y controles: ASP/ASO vs Control

## Demografía

Aquí nos encontramos variables que son cuantitativas, que se analizarán con las funciones automatizadas, y variables cualitativas, donde se realizará una tabla de contingencia y el análisis correspondiente.

Cuando las frecuencias teóricas incluyen frecuencias que son inferiores a 5, o cuando las sumas marginales del conjunto de datos (sumas por fila o por columna) son muy desiguales, es mejor confiar en la prueba exacta de Fisher.

### Variables Cualitativas

In [55]:
# variable para guardar los resultados
resultados_cualitativa = []

# selección de datos
df1 = df.copy()

df2 = df1.replace(['Add Sex', 'Add Sex Online'], 'Add Sex')

data = df2[(df['GRUPO'] == 'Add Sex') | (df['GRUPO'] == 'Control')]

**Orientación sexual**

In [56]:
# tabla contingencia 'ORSEX'
cross = pd.crosstab(df2['GRUPO'], 
            df2['ORSEX'], margins = False)
cross

ORSEX,Bisexual,Heterosexual,Homosexual
GRUPO,,,
Add Sex,3,62,15
Control,0,22,3


In [57]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}    df: {}'.format(stat, p, dof))

resultados_cualitativa.append(['Orientación sexual', 'chi2',stat, p])

Stat: 1.70625   p-valor: 0.42608134510111184    df: 2


### Si no trabaja, ¿Por qué motivo?

In [58]:
# tabla contingencia 'PANOC'
cross = pd.crosstab(df2['GRUPO'], 
            df2['PANOC'], margins = False)
cross

PANOC,baja por enfermedad,jubilado (incluye anticipada),no ocupación,paro
GRUPO,,,,
Add Sex,6,7,2,6


No hay respuesta por parte de los controles.

### Estado civil

In [59]:
# tabla contingencia 'PANOC'
cross = pd.crosstab(df2['GRUPO'], 
            df2['PAECC'], margins = False)
cross

PAECC,casado/pareja estable,separado/divorciado,soltero,viudo
GRUPO,,,,
Add Sex,52,11,16,1
Control,14,0,11,0


In [60]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}    df: {}'.format(stat, p, dof))

resultados_cualitativa.append(['Estado civil', 'chi2', stat, p])

Stat: 8.262121212121212   p-valor: 0.04089396717849641    df: 3


### ¿Tiene hijos o no?

In [61]:
# preparación de datos
boolerian = []
a = 0
for i in df2[['GRUPO', 'PAHIJ']]['PAHIJ']:
    if i > 0:
        boolerian.append('Si')
    if i == 0:
        boolerian.append('No')
    else:
        a += 1

newdata = pd.DataFrame(zip(df2[['GRUPO', 'PAHIJ']]['GRUPO'], boolerian))

# tabla contigencia
cross = pd.crosstab(newdata[0], 
            newdata[1], margins = False)

cross

1,No,Si
0,,
Add Sex,39,41
Control,12,12


In [62]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}'.format(stat, p))

resultados_cualitativa.append(['¿Hijos?', 'chi2', stat, p])

Stat: 0.015710938463435686   p-valor: 0.9002517878883406


### Trabaja

In [63]:
# tabla contigencia 'PATRT'
cross = pd.crosstab(df2['GRUPO'], 
            df2['PATRT'], margins = False)
cross

PATRT,no,si
GRUPO,,
Add Sex,25,55
Control,11,14


In [64]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}'.format(stat, p))

resultados_cualitativa.append(['¿Trabaja?', 'chi2', stat, p])

Stat: 0.8666779891304345   p-valor: 0.35187662632858885


### Enfermedades actuales

In [65]:
# tabla contigencia 'PAENF'
cross = pd.crosstab(df2['GRUPO'], 
            df2['PAENF'], margins = False)
cross

PAENF,no,si
GRUPO,,
Add Sex,53,26
Control,24,1


In [66]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}'.format(stat, p))

resultados_cualitativa.append(['¿Otra enfermedad?', 'chi2', stat, p])

Stat: 6.822545892925641   p-valor: 0.009001415872919614


### Presenta abuso de alcohol

In [67]:
# tabla contigencia 'PAALC'
cross = pd.crosstab(df2['GRUPO'], 
            df2['PAALC'], margins = False)
cross

PAALC,no,sí
GRUPO,,
Add Sex,76,4
Control,9,0


In [68]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}'.format(stat, p))

resultados_cualitativa.append(['¿Abuso alcohol?', 'chi2', stat, p])

Stat: 0.026267361111111096   p-valor: 0.8712490875704108


### Presenta abuso de sustancias

In [69]:
# tabla contigencia 'PASUS'
cross = pd.crosstab(df2['GRUPO'], 
            df2['PASUS'], margins = False)
cross

PASUS,no,sí
GRUPO,,
Add Sex,66,14
Control,12,0


In [70]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}'.format(stat, p))

resultados_cualitativa.append(['¿Abuso sustancias?', 'chi2', stat, p])

Stat: 1.306211843711843   p-valor: 0.2530816687227829


### ¿Abuso cannabis?

In [71]:
# tabla contigencia 'PASUS01'
cross = pd.crosstab(df2['GRUPO'], 
            df2['PASUS01'], margins = False)
cross

PASUS01,no,sí
GRUPO,,
Add Sex,68,12


No hay respuesta por parte de los controles.

### ¿Abuso cocaína?

In [72]:
# tabla contigencia 'PASUS02'
cross = pd.crosstab(df2['GRUPO'], 
            df2['PASUS02'], margins = False)
cross

PASUS02,no,sí
GRUPO,,
Add Sex,75,5


No hay respuesta por parte de los controles.

### ¿Abuso heroína?

In [73]:
# tabla contigencia 'PASUS03'
cross = pd.crosstab(df2['GRUPO'], 
            df2['PASUS03'], margins = False)
cross

PASUS03,no
GRUPO,
Add Sex,80


No hay respuesta por parte de los controles.

### ¿Abuso alucinógenos?

In [74]:
# tabla contigencia 'PASUS04'
cross = pd.crosstab(df2['GRUPO'], 
            df2['PASUS04'], margins = False)
cross

PASUS04,no,sí
GRUPO,,
Add Sex,79,1


No hay respuesta por parte de los controles.

### ¿Abuso drogas de diseño?

In [75]:
# tabla contigencia 'PASUS05'
cross = pd.crosstab(df2['GRUPO'], 
            df2['PASUS05'], margins = False)
cross

PASUS05,no,sí
GRUPO,,
Add Sex,78,2


No hay respuesta por parte de los controles.

### ¿Abuso de otras drogas?

In [76]:
# tabla contigencia 'PASUS06'
cross = pd.crosstab(df2['GRUPO'], 
            df2['PASUS06'], margins = False)
cross

PASUS06,no,sí
GRUPO,,
Add Sex,78,2


No hay respuesta por parte de los controles.

### ¿Ha habido intentos de autolíticos?

In [77]:
# preparación de datos 'PAAUTO'
boolerian = []
a = 0
for i in df2[['GRUPO', 'PAAUTO']]['PAAUTO']:
    if i > 0:
        boolerian.append('Si')
    if i == 0:
        boolerian.append('No')
    else:
        a += 1

newdata = pd.DataFrame(zip(df2[['GRUPO', 'PAAUTO']]['GRUPO'], boolerian))

# tabla contigencia
cross = pd.crosstab(newdata[0], 
            newdata[1], margins = False)

cross

1,No,Si
0,,
Add Sex,72,8
Control,25,0


In [78]:
# test fisher
oddsratio, p = stats.fisher_exact(cross)

print('Stat: {}   p-valor: {}'.format(oddsratio, p))

resultados_cualitativa.append(['¿Ha habido intentos de autolíticos?', 'fisher', oddsratio, p])

Stat: 0.0   p-valor: 0.19352656952428182


### Nivel de estudios

In [79]:
# tabla contigencia 'studies'
cross = pd.crosstab(df2['GRUPO'], 
            df2['studies'], margins = False)
cross

studies,No estudios,Primarios,Secundarios,Universitarios
GRUPO,,,,
Add Sex,1,26,20,33
Control,0,4,9,12


In [80]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}    df: {}'.format(stat, p, dof))

resultados_cualitativa.append(['Nivel de estudios', 'chi2', stat, p])

Stat: 3.16448275862069   p-valor: 0.36695378541693846    df: 3


### ¿Tiene empleo?

In [81]:
# tabla contigencia 'employed'
cross = pd.crosstab(df2['GRUPO'], 
            df2['employed'], margins = False)
cross

employed,No,Sí
GRUPO,,
Add Sex,23,57
Control,11,14


In [82]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}'.format(stat, p))

resultados_cualitativa.append(['¿Tiene empleo?', 'chi2', stat, p])

Stat: 1.3865795878210443   p-valor: 0.2389835756448648


### ¿Es fumador?

In [83]:
# tabla contigencia 'tabaco'
cross = pd.crosstab(df2['GRUPO'], 
            df2['tabaco'], margins = False)
cross

tabaco,No,Sí
GRUPO,,
Add Sex,57,23


No hay respuesta por parte de los controles.

**Resultados variables cualitativas.**

In [84]:
output = pd.DataFrame(resultados_cualitativa, columns = ['Variable', 'Prueba', 'Stat', 'p-valor'])

output.to_excel("ASvsControl_demografía_variables_cualitativas.xlsx")
output

,Variable,Prueba,Stat,p-valor
0,Orientación sexual,chi2,1.706250,0.426081
1,Estado civil,chi2,8.262121,0.040894
2,¿Hijos?,chi2,0.015711,0.900252
3,¿Trabaja?,chi2,0.866678,0.351877
4,¿Otra enfermedad?,chi2,6.822546,0.009001
5,¿Abuso alcohol?,chi2,0.026267,0.871249
6,¿Abuso sustancias?,chi2,1.306212,0.253082
7,¿Ha habido intentos de autolíticos?,fisher,0.000000,0.193527
8,Nivel de estudios,chi2,3.164483,0.366954
9,¿Tiene empleo?,chi2,1.386580,0.238984


In [85]:
# variables a analizar
var_analisis = ['PAEDA', 'PAHIJ']

output = pd.DataFrame(estudio_completo(df2, 'Add Sex', 'Control', var_analisis), 
             index = var_analisis, columns = columns_names)

output.to_excel("ASPvsControl_demografía_variables_cuantitativas.xlsx")
output

,grupo_uno,tamA,meanA,sdA,ic95A,IC95A,grupo_dos,tamB,meanB,sdB,ic95B,IC95B,dist,tipo de test,stat_prueba,p-valor,d Cohen
PAEDA,Add Sex,80,42.912500,11.131255,40.419732,45.405268,Control,25,43.80,16.716459,36.757497,50.842503,False,Mann-Whitney U Test,975.0,0.426841,95.150257
PAHIJ,Add Sex,79,0.924051,1.028359,0.692239,1.155862,Control,25,0.92,1.055272,0.475423,1.364577,False,Mann-Whitney U Test,977.0,0.467341,95.802732


In [86]:
df2

,CASO,PANHC,PASEX,PAEDA,ORSEX,GRUPO,PAADI03,PAADI05,PANAC,PATRT,...,PSDI,BIScognitiva,BISmotor,BISnoplaneada,TotalBIS,TOTALECS,INTER,FALLOIMP,AE,AR
0,7.0,142473.0,Home,49.0,Heterosexual,Add Sex,sí,no,BCN,si,...,1.364706,16.0,16.0,18.0,50.0,NaN,NaN,NaN,29.0,30.0
1,24.0,142482.0,Home,55.0,Heterosexual,Add Sex,sí,no,bcn,no,...,1.833333,4.0,10.0,12.0,26.0,NaN,NaN,NaN,0.0,4.0
2,30.0,142462.0,Home,34.0,Heterosexual,Add Sex,sí,sí,Cataluña. Tarragona,si,...,1.836735,2.0,3.0,13.0,18.0,NaN,NaN,NaN,21.0,16.0
3,37.0,152508.0,Home,20.0,Heterosexual,Add Sex,sí,no,bcn,si,...,1.368421,13.0,13.0,15.0,41.0,NaN,NaN,NaN,34.0,20.0
4,43.0,81822.0,Home,35.0,Homosexual,Add Sex,sí,no,bcn,si,...,1.654545,17.0,23.0,14.0,54.0,27.0,15.0,12.0,25.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,70.0,NaN,Home,61.0,Heterosexual,Control,no,no,BCN,no,...,1.266667,12.0,6.0,9.0,27.0,15.0,7.0,8.0,8.0,11.0
101,71.0,NaN,Home,26.0,Homosexual,Control,no,no,Toledo,si,...,1.314815,26.0,29.0,35.0,90.0,18.0,8.0,10.0,40.0,23.0
102,72.0,NaN,Home,30.0,Homosexual,Control,no,no,cANARIAS,si,...,1.270270,21.0,14.0,25.0,60.0,11.0,5.0,6.0,18.0,16.0
103,73.0,NaN,Home,30.0,Heterosexual,Control,no,no,bcn,no,...,1.200000,11.0,1.0,9.0,21.0,10.0,5.0,5.0,9.0,6.0


## Personalidad

Las variables que se van a comparar son las siguientes: TCI pre: Novelty Seeking, TCI pre: Harm Avoidance, TCI pre: Reguard Dependence, TCI pre: Persistence, TCI pre: Self Directness, TCI pre: Cooperation, TCI pre: Autotransendence, TCI pre: validity

In [87]:
# unión de los pacientes en una sóla 
df1 = df.replace(['Add Sex', 'Add Sex Online'], 'Add Sex')

In [88]:
# datos de la personalidad
df1[['GRUPO', 'NSTOTAL', 'HATOTAL', 
    'RDTOTAL', 'PSTOTAL', 
    'SDTOTAL', 'CTOTAL', 
    'STTOTAL', 'VALID']]

,GRUPO,NSTOTAL,HATOTAL,RDTOTAL,PSTOTAL,SDTOTAL,CTOTAL,STTOTAL,VALID
0,Add Sex,101.0,106.0,99.0,124.0,130.0,137.0,71.0,15.0
1,Add Sex,107.0,73.0,114.0,151.0,176.0,148.0,50.0,15.0
2,Add Sex,81.0,91.0,125.0,116.0,171.0,159.0,75.0,15.0
3,Add Sex,95.0,110.0,88.0,97.0,146.0,132.0,85.0,15.0
4,Add Sex,102.0,137.0,89.0,111.0,116.0,113.0,48.0,15.0
...,...,...,...,...,...,...,...,...,...
100,Control,89.0,114.0,82.0,103.0,162.0,143.0,51.0,15.0
101,Control,130.0,114.0,98.0,89.0,117.0,124.0,56.0,15.0
102,Control,112.0,111.0,124.0,102.0,131.0,151.0,66.0,15.0
103,Control,94.0,84.0,115.0,107.0,179.0,157.0,45.0,15.0


In [89]:
# análisis items de test personalidad
var_analisis = ['NSTOTAL', 'HATOTAL', 
                'RDTOTAL', 'PSTOTAL', 
                'SDTOTAL', 'CTOTAL', 
                'STTOTAL', 'VALID']


output = pd.DataFrame(estudio_completo(df1, 'Add Sex', 'Control', var_analisis), 
             index = var_analisis, columns = columns_names)

output.to_excel("ASvsControl_personalidad.xlsx")
output

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:2098: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:2099: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


,grupo_uno,tamA,meanA,sdA,ic95A,IC95A,grupo_dos,tamB,meanB,sdB,ic95B,IC95B,dist,tipo de test,stat_prueba,p-valor,d Cohen
NSTOTAL,Add Sex,80,105.2500,15.966762,101.674353,108.825647,Control,25,99.16,13.613758,93.424639,104.895361,True,t de student,1.705062,9.119733e-02,0.410461
HATOTAL,Add Sex,80,104.6500,21.382879,99.861451,109.438549,Control,25,91.88,15.618758,85.299950,98.460050,True,t de welch,3.197291,2.324129e-03,0.682014
RDTOTAL,Add Sex,80,99.1250,14.196280,95.945840,102.304160,Control,25,104.08,13.071863,98.572935,109.587065,True,t de student,-1.536828,1.274019e-01,-0.363119
PSTOTAL,Add Sex,80,107.2875,19.847666,102.842751,111.732249,Control,25,106.36,21.109012,97.466952,115.253048,True,t de student,0.198917,8.427197e-01,0.045270
SDTOTAL,Add Sex,80,124.8250,23.054162,119.662179,129.987821,Control,25,158.68,17.428069,151.337702,166.022298,True,t de student,-6.698640,1.138716e-09,-1.656663
CTOTAL,Add Sex,80,131.4375,18.742228,127.240307,135.634693,Control,25,141.36,12.601206,136.051219,146.668781,False,Mann-Whitney U Test,639.500000,3.369921e-03,62.408810
STTOTAL,Add Sex,80,67.2250,15.150887,63.832063,70.617937,Control,25,52.28,11.315547,47.512856,57.047144,False,Mann-Whitney U Test,428.000000,8.476198e-06,41.768523
VALID,Add Sex,80,14.9875,0.249687,14.931584,15.043416,Control,25,15.00,0.000000,NaN,NaN,False,Mann-Whitney U Test,1000.000000,4.936622e-01,97.590007


<font size= 1.5, color=red>**NOTA**: El error de devuelve python es debido a que el ítem **VALID** tiene valor 15 para todos los pacientes, a excepción de un paciente adicto al sexo online, que tiene valor 13. El error es consecuencia del cálculo de los índices de confianza 95% de los adictos al sexo presencial, ya que todos sus valores son 15 y no es posible calcular el IC95.</font>

Vemos que existen diferencias en cada uno de los ítems de la personalidad cuando comparamos AS y Controles.

## Psicopatología

Análisis de plas variables de **Escala de Compulsividad Sexual (SCS) (Kalichman et al., 1994)**.

In [90]:
# análisis items 'Symptom CheckList-90 items-Revised (Derogatis, 1990)'
var_analisis = ['SOMATP', 'OBSESP', 'SENSIP', 'DEPRESP', 
                'ANSIEP', 'HOSTILP', 'FOBIAP', 'PARANP', 
                'PSICOTP', 'GSI', 'PSDI']

output = pd.DataFrame(estudio_completo(df1, 'Add Sex', 'Control', var_analisis), 
             index = var_analisis, columns = columns_names)

output.to_excel("ASvsControl_SCL.xlsx")
output

,grupo_uno,tamA,meanA,sdA,ic95A,IC95A,grupo_dos,tamB,meanB,sdB,ic95B,IC95B,dist,tipo de test,stat_prueba,p-valor,d Cohen
SOMATP,Add Sex,80,0.994792,0.755115,0.825689,1.163894,Control,25,0.323333,0.306703,0.194122,0.452545,False,Mann-Whitney U Test,432.000000,9.385081e-06,42.158883
OBSESP,Add Sex,80,1.480000,0.786511,1.303866,1.656134,Control,25,0.448000,0.339553,0.304949,0.591051,True,t de welch,9.181269,1.221007e-14,1.703638
SENSIP,Add Sex,80,1.275000,0.917083,1.069625,1.480375,Control,25,0.231111,0.247296,0.126927,0.335295,False,Mann-Whitney U Test,270.000000,1.894173e-08,26.349302
DEPRESP,Add Sex,80,1.554808,0.886260,1.356336,1.753280,Control,25,0.323077,0.290276,0.200786,0.445368,False,Mann-Whitney U Test,181.000000,3.563713e-10,17.663791
ANSIEP,Add Sex,80,1.131250,0.825166,0.946460,1.316040,Control,25,0.288000,0.281879,0.169247,0.406753,False,Mann-Whitney U Test,294.000000,5.245213e-08,28.691462
HOSTILP,Add Sex,80,1.016667,0.918407,0.810996,1.222338,Control,25,0.240000,0.368420,0.084788,0.395212,False,Mann-Whitney U Test,386.500000,1.691330e-06,37.718538
FOBIAP,Add Sex,80,0.625000,0.778334,0.450697,0.799303,Control,25,0.074286,0.140669,0.015023,0.133548,False,Mann-Whitney U Test,457.000000,1.189802e-05,44.598633
PARANP,Add Sex,80,1.122917,0.839454,0.934927,1.310907,Control,25,0.153333,0.235325,0.054193,0.252474,False,Mann-Whitney U Test,271.500000,1.675477e-08,26.495687
PSICOTP,Add Sex,80,1.240000,0.793820,1.062229,1.417771,Control,25,0.144000,0.221052,0.050873,0.237127,False,Mann-Whitney U Test,148.500000,6.904839e-11,14.492116
GSI,Add Sex,80,1.222639,0.704101,1.064960,1.380317,Control,25,0.278667,0.203862,0.192781,0.364552,False,Mann-Whitney U Test,170.000000,2.172880e-10,16.590301


Vemos que existen diferencias en cada uno de los ítems de SCL90 cuando comparamos AS y Controles.

Análisis de plas variables de **'El Inventario de Ansiedad Estado-Rasgo (STAI)'**.

In [91]:
# análisis items 'El Inventario de Ansiedad Estado-Rasgo (STAI)'
var_analisis = ['AE', 'AR']

output = pd.DataFrame(estudio_completo(df1, 'Add Sex', 'Control', var_analisis), 
             index = var_analisis, columns = columns_names)

output.to_excel("ASvsControl_psicopatologia_STAI.xlsx")
output

,grupo_uno,tamA,meanA,sdA,ic95A,IC95A,grupo_dos,tamB,meanB,sdB,ic95B,IC95B,dist,tipo de test,stat_prueba,p-valor,d Cohen
AE,Add Sex,80,23.9875,12.617937,21.161799,26.813201,Control,25,10.24,8.021371,6.860664,13.619336,False,Mann-Whitney U Test,319.500000,1.531598e-07,31.180007
AR,Add Sex,80,27.9750,11.135501,25.481281,30.468719,Control,25,13.00,6.033241,10.458247,15.541753,True,t de welch,8.524114,1.200611e-12,1.672170


Vemos que existen diferencias en cada uno de los ítems de STAI cuando comparamos AS y Controles.

Análisis de plas variables de **'The Barratt Impulsiveness Scale (BIS-11)'**.

In [92]:
# análisis items 'The Barratt Impulsiveness Scale (BIS-11)'
var_analisis = ['BIScognitiva', 'BISmotor', 'BISnoplaneada', 'TotalBIS']

output = pd.DataFrame(estudio_completo(df2, 'Add Sex', 'Control', var_analisis), 
             index = var_analisis, columns = columns_names)

output.to_excel("ASvsControl_psicopatologia_BIS11.xlsx")
output

,grupo_uno,tamA,meanA,sdA,ic95A,IC95A,grupo_dos,tamB,meanB,sdB,ic95B,IC95B,dist,tipo de test,stat_prueba,p-valor,d Cohen
BIScognitiva,Add Sex,80,16.7375,5.365966,15.535829,17.939171,Control,25,12.2,4.698936,10.220378,14.179622,False,Mann-Whitney U Test,482.500000,0.000049,47.087179
BISmotor,Add Sex,80,17.5750,7.436019,15.909755,19.240245,Control,25,12.4,7.059745,9.425789,15.374211,True,t de student,3.044213,0.002961,0.713761
BISnoplaneada,Add Sex,80,20.4500,8.002968,18.657790,22.242210,Control,25,15.0,6.663332,12.192794,17.807206,False,Mann-Whitney U Test,578.000000,0.000752,56.407024
TotalBIS,Add Sex,80,54.7625,15.756145,51.234019,58.290981,Control,25,39.6,15.694585,32.988004,46.211996,False,Mann-Whitney U Test,421.000000,0.000007,41.085393


Vemos que existen diferencias en cada uno de los ítems de BIS11 cuando comparamos AS y Controles

Análisis de plas variables de **'Escala de Compulsividad Sexual (SCS)'**.

In [93]:
# análisis items 'Escala de Compulsividad Sexual (SCS)'
var_analisis = ['TOTALECS', 'INTER', 'FALLOIMP']

output = pd.DataFrame(estudio_completo(df2, 'Add Sex', 'Control', var_analisis), 
             index = var_analisis, columns = columns_names)

output.to_excel("ASvsControl_psicopatologia_SCS.xlsx")
output

,grupo_uno,tamA,meanA,sdA,ic95A,IC95A,grupo_dos,tamB,meanB,sdB,ic95B,IC95B,dist,tipo de test,stat_prueba,p-valor,d Cohen
TOTALECS,Add Sex,74,27.337838,7.063949,25.690082,28.985594,Control,25,11.0,1.833030,10.227760,11.772240,False,Mann-Whitney U Test,12.5,8.975496e-14,1.256297
INTER,Add Sex,74,13.608108,3.664356,12.753351,14.462865,Control,25,5.4,0.748331,5.084734,5.715266,False,Mann-Whitney U Test,26.5,1.769612e-13,2.663350
FALLOIMP,Add Sex,74,14.040541,4.212078,13.058020,15.023061,Control,25,5.6,1.131371,5.123363,6.076637,False,Mann-Whitney U Test,40.5,4.365780e-13,4.070403


# Comparación entre patientes y controles: ASP vs ASO vs Control

## Demografía

## Demografía

Aquí nos encontramos variables que son cuantitativas, que se analizarán con las funciones automatizadas, y variables cualitativas, donde se realizará una tabla de contingencia y el análisis correspondiente.

Cuando las frecuencias teóricas incluyen frecuencias que son inferiores a 5, o cuando las sumas marginales del conjunto de datos (sumas por fila o por columna) son muy desiguales, es mejor confiar en la prueba exacta de Fisher.

### Variables Cualitativas

In [94]:
# variable para guardar los resultados
resultados_cualitativa = []

**Orientación sexual**

In [95]:
# tabla contingencia 'ORSEX'
data = df.copy()

cross = pd.crosstab(data['GRUPO'], 
            data['ORSEX'], margins = False)
cross

ORSEX,Bisexual,Heterosexual,Homosexual
GRUPO,,,
Add Sex,2,29,13
Add Sex Online,1,33,2
Control,0,22,3


In [96]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}   dof:{}'.format(stat, p, dof))

resultados_cualitativa.append(['Orientación sexual', 'chi2',stat, p])

Stat: 10.212037037037039   p-valor: 0.037003469836470855   dof:4


### Si no trabaja, ¿Por qué motivo?

In [97]:
# tabla contingencia 'PANOC'
cross = pd.crosstab(data['GRUPO'], 
            data['PANOC'], margins = False)
cross

PANOC,baja por enfermedad,jubilado (incluye anticipada),no ocupación,paro
GRUPO,,,,
Add Sex,3,5,1,4
Add Sex Online,3,2,1,2


Sin datos para controles

### Estado civil

In [98]:
# tabla contingencia 'PAECC'
cross = pd.crosstab(data['GRUPO'], 
            data['PAECC'], margins = False)
cross

PAECC,casado/pareja estable,separado/divorciado,soltero,viudo
GRUPO,,,,
Add Sex,25,6,12,1
Add Sex Online,27,5,4,0
Control,14,0,11,0


In [99]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}   dof:{}'.format(stat, p, dof))

resultados_cualitativa.append(['Estado civil', 'chi2', stat, p])

Stat: 12.389735741250892   p-valor: 0.05381808782275499   dof:6


### ¿Tiene hijos o no?

In [100]:
# preparación de datos
boolerian = []
a = 0
for i in data[['GRUPO', 'PAHIJ']]['PAHIJ']:
    if i > 0:
        boolerian.append('Si')
    if i == 0:
        boolerian.append('No')
    else:
        a += 1

newdata = pd.DataFrame(zip(data[['GRUPO', 'PAHIJ']]['GRUPO'], boolerian))

# tabla contigencia
cross = pd.crosstab(newdata[0], 
            newdata[1], margins = False)

cross

1,No,Si
0,,
Add Sex,24,20
Add Sex Online,15,21
Control,12,12


In [101]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}   dof:{}'.format(stat, p, dof))

resultados_cualitativa.append(['¿Hijos?', 'chi2', stat, p])

Stat: 1.3256650859314565   p-valor: 0.5153894023501193   dof:2


### Trabaja

In [102]:
# tabla contigencia 'PATRT'
cross = pd.crosstab(data['GRUPO'], 
            data['PATRT'], margins = False)
cross

PATRT,no,si
GRUPO,,
Add Sex,14,30
Add Sex Online,11,25
Control,11,14


In [103]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}   dof:{}'.format(stat, p, dof))

resultados_cualitativa.append(['¿Trabaja?', 'chi2', stat, p])

Stat: 1.388330771482945   p-valor: 0.4994911565067387   dof:2


### Enfermedades actuales

In [104]:
# tabla contigencia 'PAENF'
cross = pd.crosstab(data['GRUPO'], 
            data['PAENF'], margins = False)
cross

PAENF,no,si
GRUPO,,
Add Sex,23,20
Add Sex Online,30,6
Control,24,1


In [105]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}   dof:{}'.format(stat, p, dof))

resultados_cualitativa.append(['¿Otra enfermedad?', 'chi2', stat, p])

Stat: 17.338428806335784   p-valor: 0.00017179401205765646   dof:2


### Presenta abuso de alcohol

In [106]:
# tabla contigencia 'PAALC'
cross = pd.crosstab(data['GRUPO'], 
            data['PAALC'], margins = False)
cross

PAALC,no,sí
GRUPO,,
Add Sex,41,3
Add Sex Online,35,1
Control,9,0


In [107]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}   dof:{}'.format(stat, p, dof))

resultados_cualitativa.append(['¿Abuso alcohol?', 'chi2', stat, p])

Stat: 1.224212715389186   p-valor: 0.5422075822987154   dof:2


### Presenta abuso de sustancias

In [108]:
# tabla contigencia 'PASUS'
cross = pd.crosstab(data['GRUPO'], 
            data['PASUS'], margins = False)
cross

PASUS,no,sí
GRUPO,,
Add Sex,35,9
Add Sex Online,31,5
Control,12,0


In [109]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}   dof:{}'.format(stat, p, dof))

resultados_cualitativa.append(['¿Abuso sustancias?', 'chi2', stat, p])

Stat: 3.1384911384911374   p-valor: 0.20820219726189113   dof:2


### ¿Abuso cannabis?

In [110]:
# tabla contigencia 'PASUS01'
cross = pd.crosstab(data['GRUPO'], 
            data['PASUS01'], margins = False)
cross

PASUS01,no,sí
GRUPO,,
Add Sex,37,7
Add Sex Online,31,5


Sin resultados para controles.

### ¿Abuso cocaína?

In [111]:
# tabla contigencia 'PASUS02'
cross = pd.crosstab(data['GRUPO'], 
            data['PASUS02'], margins = False)
cross

PASUS02,no,sí
GRUPO,,
Add Sex,39,5
Add Sex Online,36,0


Sin resultados para controles.

### ¿Abuso heroína?

In [112]:
# tabla contigencia 'PASUS03'
cross = pd.crosstab(data['GRUPO'], 
            data['PASUS03'], margins = False)
cross

PASUS03,no
GRUPO,
Add Sex,44
Add Sex Online,36


Sin resultados para controles.

### ¿Abuso alucinógenos?

In [113]:
# tabla contigencia 'PASUS04'
cross = pd.crosstab(data['GRUPO'], 
            data['PASUS04'], margins = False)
cross

PASUS04,no,sí
GRUPO,,
Add Sex,43,1
Add Sex Online,36,0


Sin resultados para controles.

### ¿Abuso drogas de diseño?

In [114]:
# tabla contigencia 'PASUS05'
cross = pd.crosstab(data['GRUPO'], 
            data['PASUS05'], margins = False)
cross

PASUS05,no,sí
GRUPO,,
Add Sex,42,2
Add Sex Online,36,0


Sin resultados para controles.

### ¿Abuso de otras drogas?

In [115]:
# tabla contigencia 'PASUS06'
cross = pd.crosstab(data['GRUPO'], 
            data['PASUS06'], margins = False)
cross

PASUS06,no,sí
GRUPO,,
Add Sex,42,2
Add Sex Online,36,0


Sin resultados para controles.

### ¿Ha habido intentos de autolíticos?

In [116]:
# preparación de datos 'PAAUTO'
boolerian = []
a = 0
for i in data[['GRUPO', 'PAAUTO']]['PAAUTO']:
    if i > 0:
        boolerian.append('Si')
    if i == 0:
        boolerian.append('No')
    else:
        a += 1

newdata = pd.DataFrame(zip(data[['GRUPO', 'PAAUTO']]['GRUPO'], boolerian))

# tabla contigencia
cross = pd.crosstab(newdata[0], 
            newdata[1], margins = False)

cross

1,No,Si
0,,
Add Sex,40,4
Add Sex Online,32,4
Control,25,0


In [117]:
# test fisher
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}   dof:{}'.format(stat, p, dof))

resultados_cualitativa.append(['¿Ha habido intentos de autolíticos?', 'fisher', oddsratio, p])

Stat: 2.820993439550141   p-valor: 0.24402204247055934   dof:2


### Nivel de estudios

In [118]:
# tabla contigencia 'studies'
cross = pd.crosstab(data['GRUPO'], 
            data['studies'], margins = False)
cross

studies,No estudios,Primarios,Secundarios,Universitarios
GRUPO,,,,
Add Sex,1,15,11,17
Add Sex Online,0,11,9,16
Control,0,4,9,12


In [119]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}   dof:{}'.format(stat, p, dof))

resultados_cualitativa.append(['Nivel de estudios', 'chi2', stat, p])

Stat: 4.480812724950656   p-valor: 0.6119001197913729   dof:6


### ¿Tiene empleo?

In [120]:
# tabla contigencia 'employed'
cross = pd.crosstab(data['GRUPO'], 
            data['employed'], margins = False)
cross

employed,No,Sí
GRUPO,,
Add Sex,14,30
Add Sex Online,9,27
Control,11,14


In [121]:
# test chi cuadrado
stat, p, dof, expected = stats.chi2_contingency(cross)

print('Stat: {}   p-valor: {}'.format(stat, p))

resultados_cualitativa.append(['¿Tiene empleo?', 'chi2', stat, p])

Stat: 2.443501920614597   p-valor: 0.2947136829236608


### ¿Es fumador?

In [122]:
# tabla contigencia 'tabaco'
cross = pd.crosstab(data['GRUPO'], 
            data['tabaco'], margins = False)
cross

tabaco,No,Sí
GRUPO,,
Add Sex,31,13
Add Sex Online,26,10


sin resultados para controles.

**Resultados variables cualitativas.**

In [123]:
output = pd.DataFrame(resultados_cualitativa, columns = ['Variable', 'Prueba', 'Stat', 'p-valor'])

output.to_excel("ASPvsASOvsControles_demografía_variables_cualitativas.xlsx")
output

,Variable,Prueba,Stat,p-valor
0,Orientación sexual,chi2,10.212037,0.037003
1,Estado civil,chi2,12.389736,0.053818
2,¿Hijos?,chi2,1.325665,0.515389
3,¿Trabaja?,chi2,1.388331,0.499491
4,¿Otra enfermedad?,chi2,17.338429,0.000172
5,¿Abuso alcohol?,chi2,1.224213,0.542208
6,¿Abuso sustancias?,chi2,3.138491,0.208202
7,¿Ha habido intentos de autolíticos?,fisher,0.000000,0.244022
8,Nivel de estudios,chi2,4.480813,0.611900
9,¿Tiene empleo?,chi2,2.443502,0.294714


Se observan diferencias estadisticamente significativas en las variables: orientación sexual y presenta otras enfermedades.

In [124]:
# variables a analizar
var_analisis = ['PAEDA', 'PAHIJ']

#output = pd.DataFrame(estudio_completo_3g(df, 'GRUPO', 'Add Sex', 'Add Sex Online', 'Control', var_analisis), 
#             index = var_analisis, columns = columns_names2)

#output.to_excel("ASPvsASOvsControl_demografía_variables_cuantitativas.xlsx")
#output

ANOVA_3grupos(df, 'GRUPO', 'PAEDA')

,sum_sq,df,F,PR(>F)
C(GRUPO),43.731385,2.0,0.132208,0.876309
Residual,16869.659091,102.0,NaN,NaN


## Personalidad

In [144]:
# datos de la personalidad
var_analisis = ['NSTOTAL', 'HATOTAL', 'RDTOTAL', 'PSTOTAL', 'SDTOTAL', 'CTOTAL','STTOTAL', 'VALID']

output = pd.DataFrame(estudio_completo_3g(df, 'GRUPO', 'Add Sex', 'Add Sex Online', 'Control', var_analisis), columns = columns_names2)

#output.to_excel("ASPvsASOvsControl_personalidad.xlsx")
#output

#ANOVA_3grupos(df, 'GRUPO', var_analisis[4])

pg.kruskal(dv = 'STTOTAL', between= 'GRUPO', data = df[['GRUPO', 'STTOTAL']])

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:2098: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:2099: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\morestats.py:2303: RuntimeWarning: divide by zero encountered in log
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)


,Source,ddof1,H,p-unc
Kruskal,GRUPO,2,18.576102,0.000093


<font size= 1.5, color=red>**NOTA**: El error de devuelve python es debido a que el ítem **VALID** tiene valor 15 para todos los pacientes, a excepción de un paciente adicto al sexo online, que tiene valor 13. El error es consecuencia del cálculo de los índices de confianza 95% de los adictos al sexo presencial, ya que todos sus valores son 15 y no es posible calcular el IC95.</font>

## Psicopatología

In [126]:
# análisis items 'Symptom CheckList-90 items-Revised (Derogatis, 1990)'
var_analisis = ['SOMATP', 'OBSESP', 'SENSIP', 'DEPRESP', 
                'ANSIEP', 'HOSTILP', 'FOBIAP', 'PARANP', 
                'PSICOTP', 'GSI', 'PSDI']

output = pd.DataFrame(estudio_completo_3g(df, 'GRUPO', 'Add Sex', 
                                          'Add Sex Online', 'Control', var_analisis), columns = columns_names2)

output.to_excel("ASPvsASOvsControl_SCL.xlsx")
output

,Variable,grupo experimental 1,tamaño,media,SD,IC.05,IC.95,grupo experimental 2,tamaño,media,...,grupo experimental 3,tamaño,media,SD,IC.05,IC.95,dis,tipo de test,p-valor,d Cohen
0,SOMATP,Add Sex,44,1.047348,0.845815,0.787224,1.307473,Add Sex Online,36,0.930556,...,Control,25,0.323333,0.306703,0.194122,0.452545,False,kruskal-wallis,1.030095e-04,-0.267234
1,OBSESP,Add Sex,44,1.502273,0.794152,1.258037,1.746508,Add Sex Online,36,1.452778,...,Control,25,0.448000,0.339553,0.304949,0.591051,True,Welch ANOVA,2.101830e-12,-0.485321
2,SENSIP,Add Sex,44,1.371212,0.998580,1.064106,1.678318,Add Sex Online,36,1.157407,...,Control,25,0.231111,0.247296,0.126927,0.335295,False,kruskal-wallis,2.194672e-07,-0.018864
3,DEPRESP,Add Sex,44,1.617133,0.948582,1.325403,1.908862,Add Sex Online,36,1.478632,...,Control,25,0.323077,0.290276,0.200786,0.445368,False,kruskal-wallis,5.091927e-09,-0.205135
4,ANSIEP,Add Sex,44,1.331818,0.895893,1.056293,1.607343,Add Sex Online,36,0.886111,...,Control,25,0.288000,0.281879,0.169247,0.406753,False,kruskal-wallis,1.120862e-07,0.195241
5,HOSTILP,Add Sex,44,1.208333,1.051169,0.885054,1.531613,Add Sex Online,36,0.782407,...,Control,25,0.240000,0.368420,0.084788,0.395212,False,kruskal-wallis,7.204637e-06,0.203786
6,FOBIAP,Add Sex,44,0.639610,0.808976,0.390816,0.888405,Add Sex Online,36,0.607143,...,Control,25,0.074286,0.140669,0.015023,0.133548,False,kruskal-wallis,1.227062e-04,-0.053543
7,PARANP,Add Sex,44,1.250000,0.879408,0.979544,1.520456,Add Sex Online,36,0.967593,...,Control,25,0.153333,0.235325,0.054193,0.252474,True,Welch ANOVA,3.086593e-12,0.153964
8,PSICOTP,Add Sex,44,1.377273,0.866514,1.110783,1.643763,Add Sex Online,36,1.072222,...,Control,25,0.144000,0.221052,0.050873,0.237127,True,Welch ANOVA,2.247549e-15,0.205267
9,GSI,Add Sex,44,1.318434,0.784766,1.077085,1.559783,Add Sex Online,36,1.105556,...,Control,25,0.278667,0.203862,0.192781,0.364552,True,Welch ANOVA,8.465382e-15,-0.093913


In [127]:
var_analisis = ['TOTALECS', 'INTER', 'FALLOIMP']

ANOVA_3grupos(df, 'GRUPO', var_analisis[2])

#pg.kruskal(dv = 'FOBIAP', between= 'GRUPO', data = df[['GRUPO', 'FOBIAP']])

,sum_sq,df,F,PR(>F)
C(GRUPO),1433.689171,2.0,55.386308,1.012405e-16
Residual,1242.492647,96.0,NaN,NaN


In [128]:
# análisis items 'El Inventario de Ansiedad Estado-Rasgo (STAI)'
var_analisis = ['AE', 'AR']

output = pd.DataFrame(estudio_completo_3g(df, 'GRUPO', 'Add Sex', 
                                          'Add Sex Online', 'Control', var_analisis), columns = columns_names2)

output.to_excel("ASPvsASOvsControl_psicopatologia_STAI.xlsx")
output

,Variable,grupo experimental 1,tamaño,media,SD,IC.05,IC.95,grupo experimental 2,tamaño,media,...,grupo experimental 3,tamaño,media,SD,IC.05,IC.95,dis,tipo de test,p-valor,d Cohen
0,AE,Add Sex,44,24.863636,14.554898,20.387384,29.339888,Add Sex Online,36,22.916667,...,Control,25,10.24,8.021371,6.860664,13.619336,True,Welch ANOVA,1.331255e-07,-0.610553
1,AR,Add Sex,44,29.363636,11.004319,25.979339,32.747934,Add Sex Online,36,26.277778,...,Control,25,13.00,6.033241,10.458247,15.541753,True,Welch ANOVA,1.572555e-11,-0.838187


In [129]:
# análisis items 'The Barratt Impulsiveness Scale (BIS-11)'
var_analisis = ['BIScognitiva', 'BISmotor', 'BISnoplaneada', 'TotalBIS']

output = pd.DataFrame(estudio_completo_3g(df, 'GRUPO', 'Add Sex', 
                                          'Add Sex Online', 'Control', var_analisis), columns = columns_names2)

output.to_excel("ASPvsASOvsControl_psicopatologia_BIS11.xlsx")
output

,Variable,grupo experimental 1,tamaño,media,SD,IC.05,IC.95,grupo experimental 2,tamaño,media,...,grupo experimental 3,tamaño,media,SD,IC.05,IC.95,dis,tipo de test,p-valor,d Cohen
0,BIScognitiva,Add Sex,44,16.750000,5.257570,15.133073,18.366927,Add Sex Online,36,16.722222,...,Control,25,12.2,4.698936,10.220378,14.179622,True,ANOVA,0.001411,-1.925528
1,BISmotor,Add Sex,44,19.000000,6.928203,16.869282,21.130718,Add Sex Online,36,15.833333,...,Control,25,12.4,7.059745,9.425789,15.374211,True,ANOVA,0.002005,-1.043683
2,BISnoplaneada,Add Sex,44,19.909091,7.558313,17.584587,22.233595,Add Sex Online,36,21.111111,...,Control,25,15.0,6.663332,12.192794,17.807206,True,ANOVA,0.009419,-1.740855
3,TotalBIS,Add Sex,44,55.659091,14.001531,51.353023,59.965159,Add Sex Online,36,53.666667,...,Control,25,39.6,15.694585,32.988004,46.211996,True,ANOVA,0.000307,-1.939138


In [130]:
# análisis items 'Escala de Compulsividad Sexual (ECS)'
var_analisis = ['TOTALECS', 'INTER', 'FALLOIMP']

output = pd.DataFrame(estudio_completo_3g(df, 'GRUPO', 'Add Sex', 
                                          'Add Sex Online', 'Control', var_analisis), columns = columns_names2)

output.to_excel("ASPvsASOvsControl_psicopatologia_ECS.xlsx")
output

,Variable,grupo experimental 1,tamaño,media,SD,IC.05,IC.95,grupo experimental 2,tamaño,media,...,grupo experimental 3,tamaño,media,SD,IC.05,IC.95,dis,tipo de test,p-valor,d Cohen
0,TOTALECS,Add Sex,40,28.750,7.234466,NaN,NaN,Add Sex Online,34,25.676471,...,Control,25,11.0,1.833030,10.227760,11.772240,True,Welch ANOVA,3.810339e-24,-1.134542
1,INTER,Add Sex,40,14.175,3.885148,NaN,NaN,Add Sex Online,34,12.941176,...,Control,25,5.4,0.748331,5.084734,5.715266,True,Welch ANOVA,1.831651e-23,-1.148925
2,FALLOIMP,Add Sex,40,15.125,4.166458,NaN,NaN,Add Sex Online,34,12.764706,...,Control,25,5.6,1.131371,5.123363,6.076637,True,Welch ANOVA,1.390969e-21,-0.787840


In [131]:
output = df.groupby('GRUPO')['PATRE'].value_counts()
output.to_excel('ASPvsASOvsControl_enfermedades.xlsx')

In [132]:
# cáculo del tamaño de muestra para ECS
from statsmodels.stats.power import tt_ind_solve_power

# datos
alpha = 0.05 # nivel de significacia
beta = 0.20
power = 1 - beta # el poder estadístico
mean_diff = 0.2 # diferencias entre medias


n = tt_ind_solve_power(effect_size = mean_diff / 1,
                       alpha = alpha,
                       power = power,
                       ratio = 1,
                       alternative='two-sided')

print('Por cada grupo: {:.5f}'.format(n))


Por cada grupo: 393.40570


In [133]:
# cáculo del tamaño de muestra para ECS
from statsmodels.stats.power import tt_ind_solve_power

# fórmula: n = [(Zα/2 + Zβ)^2 × {2(ó)^2}]/ (μ1 - μ2)^2

sigma = np.linspace(0.01, 1, 70)
diff_means = 0.2


for i in sigma:
    n = ((stats.norm.ppf(1 - 0.05) + stats.norm.ppf(1 - 0.02))**2 * (i*2)**2) / (diff_means)**2
    print('Con una diferencias entre medias de {} y con un sigma de {:.3f}, se obtiene uns SD de {:.3f}'.format(diff_means, i, n))

Con una diferencias entre medias de 0.2 y con un sigma de 0.010, se obtiene uns SD de 0.137
Con una diferencias entre medias de 0.2 y con un sigma de 0.024, se obtiene uns SD de 0.811
Con una diferencias entre medias de 0.2 y con un sigma de 0.039, se obtiene uns SD de 2.048
Con una diferencias entre medias de 0.2 y con un sigma de 0.053, se obtiene uns SD de 3.849
Con una diferencias entre medias de 0.2 y con un sigma de 0.067, se obtiene uns SD de 6.213
Con una diferencias entre medias de 0.2 y con un sigma de 0.082, se obtiene uns SD de 9.140
Con una diferencias entre medias de 0.2 y con un sigma de 0.096, se obtiene uns SD de 12.630
Con una diferencias entre medias de 0.2 y con un sigma de 0.110, se obtiene uns SD de 16.683
Con una diferencias entre medias de 0.2 y con un sigma de 0.125, se obtiene uns SD de 21.300
Con una diferencias entre medias de 0.2 y con un sigma de 0.139, se obtiene uns SD de 26.480
Con una diferencias entre medias de 0.2 y con un sigma de 0.153, se obtiene 

In [134]:
# cáculo del tamaño de muestra para SCL90
from statsmodels.stats.power import tt_ind_solve_power

# fórmula: n = [Zα/2 + Zβ)^2 × 2*(ó)^2]/ (μ1 - μ2)^2

sigma = np.linspace(0.5, 0.7, 10)
diff_means = 0.2


for i in sigma:
    n = ((stats.norm.ppf(1-0.05) + stats.norm.ppf(1-0.02))**2 * 2*(i)**2) / (diff_means)**2
    print('Con una diferencias entre medias de {} y con un sigma de {:.3f}, se obtiene un tamaño muestral de {:.3f}'.format(diff_means, i, n))

Con una diferencias entre medias de 0.2 y con un sigma de 0.500, se obtiene un tamaño muestral de 170.996
Con una diferencias entre medias de 0.2 y con un sigma de 0.522, se obtiene un tamaño muestral de 186.533
Con una diferencias entre medias de 0.2 y con un sigma de 0.544, se obtiene un tamaño muestral de 202.746
Con una diferencias entre medias de 0.2 y con un sigma de 0.567, se obtiene un tamaño muestral de 219.635
Con una diferencias entre medias de 0.2 y con un sigma de 0.589, se obtiene un tamaño muestral de 237.199
Con una diferencias entre medias de 0.2 y con un sigma de 0.611, se obtiene un tamaño muestral de 255.438
Con una diferencias entre medias de 0.2 y con un sigma de 0.633, se obtiene un tamaño muestral de 274.353
Con una diferencias entre medias de 0.2 y con un sigma de 0.656, se obtiene un tamaño muestral de 293.944
Con una diferencias entre medias de 0.2 y con un sigma de 0.678, se obtiene un tamaño muestral de 314.210
Con una diferencias entre medias de 0.2 y con 

In [135]:
scl90 = [1.64, 1.35, 1.38, 1.29, 1.34, 1.16, 1.42, 0.9, 0.81]

np.mean(scl90)

1.2544444444444445

In [136]:
var_analisis = ['CTOTAL','STTOTAL', 'VALID']


print(ANOVA_3grupos(df, 'GRUPO', 'PAEDA'))

                sum_sq     df         F    PR(>F)
C(GRUPO)     43.731385    2.0  0.132208  0.876309
Residual  16869.659091  102.0       NaN       NaN


In [137]:
ssq = 43.731385
sse = 16869.659091 

In [138]:
n = sqrt((ssq)/(ssq + sse))

In [139]:
n

0.050848866730957135

In [140]:
a = 0.5357218860680564
b = 0.7319302467230442

In [141]:
b**2

0.5357218860680564

In [142]:
0.4**2

0.16000000000000003